# 中古車価格の価格変動を調査

中古車の価格がどのように変動するのかを調査します。  
まずは必要なライブラリのインポート、その他初期設定を行います。

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import requests
from bs4 import BeautifulSoup
import re
from time import sleep
from retry import retry
from IPython.display import HTML
import random

def hide_toggle(for_next=False):
    this_cell = """$('div.cell.code_cell.rendered.selected')"""
    next_cell = this_cell + '.next()'

    toggle_text = 'Toggle show/hide'  # text shown on toggle link
    target_cell = this_cell  # target cell to control with toggle
    js_hide_current = ''  # bit of JS to permanently hide code in current cell (only when toggling next cell)

    if for_next:
        target_cell = next_cell
        toggle_text += ' next cell'
        js_hide_current = this_cell + '.find("div.input").hide();'

    js_f_name = 'code_toggle_{}'.format(str(random.randint(1,2**64)))

    html = """
        <script>
            function {f_name}() {{
                {cell_selector}.find('div.input').toggle();
            }}

            {js_hide_current}
        </script>

        <a href="javascript:{f_name}()">{toggle_text}</a>
    """.format(
        f_name=js_f_name,
        cell_selector=target_cell,
        js_hide_current=js_hide_current, 
        toggle_text=toggle_text
    )

    return HTML(html)

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 200)

hide_toggle()

「価格.com」よりデータを取得します。  
「価格.com中古車」については、以下のリンクをご参照下さい。  
▼一覧ページ  
[価格.com (トヨタ カムリ) ](https://kakaku.com/kuruma/used/spec/Maker=1/Model=30042/Page=1/)  
▼詳細ページ  
[価格.com (トヨタ カムリ WSレザーパッケージ)](https://kakaku.com/kuruma/used/item/26671455/)

スクレイピングを実行します。  
今回は価格変動が著しい国産スポーツカーを対象とします。

In [106]:
@retry(tries=3, delay=5, backoff=2)
def get_html(url):
    # 一覧ページのhtml取得
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    return soup


def parse_car_detail_info(car_url):
    # 変数定義
    data = {}
    
    # html取得
    soup2 = get_html(car_url)
    
    # 名称
    data["名称"] = soup2.find("h3").getText().strip()
 
    # 価格　
    data["価格"] = soup2.find("span", {"class": "priceTxt"}).find("span").getText().strip()
            
    # スペック
    for tr in soup2.find("table", {"class": "specList"}).findAll("tr"):
        data[tr.find("th").getText().strip()] = tr.find("td").getText().strip()
 
    # 車両情報
    dt = soup2.find("div", {"class": "carinfoArea"}).findAll("dt")
    dd = soup2.find("div", {"class": "carinfoArea"}).findAll("dd")
 
    for i, value in enumerate(dt):
        data[value.getText().strip()] = dd[i].getText().strip()
    
    # モデル
    data["モデル"] = soup2.find(class_="c-box-regularNote_item c-box-regularNote_item-nest").getText().strip()
    
    # メーカー
    data["メーカー"] = soup2.find("title").getText().strip()

    return data
 
def check_next_page(soup):
    if soup.find("li", {"class": "next"}):
        return 1
    else:
        return 0
    
    
# 変数定義
all_data = []
 
# 60ページまでしかページ遷移できないので、地域ごとに分けて収集する
url_list = [
    
    # トヨタ
    
    "https://kakaku.com/kuruma/used/spec/Maker=1/Model=30042/Page={}", #カムリ
    "https://kakaku.com/kuruma/used/spec/Maker=1/Model=30099/Page={}", #スープラ
    "https://kakaku.com/kuruma/used/spec/Maker=1/Model=33324/Page={}", #カローラスポーツ
    "https://kakaku.com/kuruma/used/spec/Maker=1/Model=30069/Prefecture=1,2,9,17/Page={}", #クラウンアスリート
    "https://kakaku.com/kuruma/used/spec/Maker=1/Model=30069/Prefecture=24,29,36,42,47/Page={}",
    "https://kakaku.com/kuruma/used/spec/Maker=1/Model=31761/Prefecture=9/Page={}", #86
    "https://kakaku.com/kuruma/used/spec/Maker=1/Model=31761/Prefecture=1,2,17,24,29,36,42,47/Page={}",
    "https://kakaku.com/kuruma/used/spec/Maker=1/Model=33416/Page={}", #GRヤリス
    "https://kakaku.com/kuruma/used/spec/Maker=1/Model=30177/Prefecture=1,2,9/Page={}", #マークX
    "https://kakaku.com/kuruma/used/spec/Maker=1/Model=30177/Prefecture=17,24,29,36,42,47/Page={}",
    
    # ホンダ
    
     "https://kakaku.com/kuruma/used/spec/Maker=2/Model=30238/Page={}/", # シビック
    "https://kakaku.com/kuruma/used/spec/Maker=2/Model=33294/Page={}", #シビックセダン
    "https://kakaku.com/kuruma/used/spec/Maker=2/Model=30241/Page={}", #シビックタイプR
    "https://kakaku.com/kuruma/used/spec/Maker=2/Model=33154/Page={}", #S660
    "https://kakaku.com/kuruma/used/spec/Maker=2/Model=30213/Page={}", #アコード
    "https://kakaku.com/kuruma/used/spec/Maker=2/Model=30204/Page={}", #NSX
    "https://kakaku.com/kuruma/used/spec/Maker=2/Model=30205/Page={}", #S2000
    "https://kakaku.com/kuruma/used/spec/Maker=2/Model=30222/Page={}", #インテグラ
    
    # 日産
    
    "https://kakaku.com/kuruma/used/spec/Maker=3/Model=30330/Prefecture=1,2,9/Page={}", #スカイライン
    "https://kakaku.com/kuruma/used/spec/Maker=3/Model=30330/Prefecture=17,24,29,36,42,47/Page={}",
    "https://kakaku.com/kuruma/used/spec/Maker=3/Model=30372/Page={}", #フーガ
    "https://kakaku.com/kuruma/used/spec/Maker=3/Model=30374/Page={}", #フェアレディZ
    "https://kakaku.com/kuruma/used/spec/Maker=3/Model=30284/Page={}", #GTR
    
    # 三菱
    
    "https://kakaku.com/kuruma/used/spec/Maker=4/Model=30493/Page={}", #ランサーエボリューション
    
    # マツダ
    
    "https://kakaku.com/kuruma/used/spec/Maker=5/Model=33354/Page={}", #mazda3
    "https://kakaku.com/kuruma/used/spec/Maker=5/Model=30516/Page={}", #アクセラ
    "https://kakaku.com/kuruma/used/spec/Maker=5/Model=30517/Page={}", #アクセラスポーツ
    "https://kakaku.com/kuruma/used/spec/Maker=5/Model=30575/Page={}", #ロードスター
    "https://kakaku.com/kuruma/used/spec/Maker=5/Model=33250/Page={}", #ロードスターRF
    "https://kakaku.com/kuruma/used/spec/Maker=5/Model=30514/Page={}", #RX-7
    "https://kakaku.com/kuruma/used/spec/Maker=5/Model=30515/Page={}", #RX-8
    
    # スバル
    
    "https://kakaku.com/kuruma/used/spec/Maker=6/Model=33125/Page={}", #WRXS4
    "https://kakaku.com/kuruma/used/spec/Maker=6/Model=31675/Page={}", #WRXSTI
    "https://kakaku.com/kuruma/used/spec/Maker=6/Model=31823/Page={}", #WRXSTI5door
    "https://kakaku.com/kuruma/used/spec/Maker=6/Model=33084/Prefecture=1,2,9/Page={}", #レヴォーグ
    "https://kakaku.com/kuruma/used/spec/Maker=6/Model=33084/Prefecture=17,24,29,36,42,47/Page={}",
    "https://kakaku.com/kuruma/used/spec/Maker=6/Model=30609/Page={}", #レガシィB4
    "https://kakaku.com/kuruma/used/spec/Maker=6/Model=31805/Prefecture=1,2,9/Page={}", #インプレッサスポーツ
    "https://kakaku.com/kuruma/used/spec/Maker=6/Model=31805/Prefecture=17,24,29,36,42,47/Page={}",
    "https://kakaku.com/kuruma/used/spec/Maker=6/Model=31762/Page={}", #BRZ
    
    # レクサス
    
    "https://kakaku.com/kuruma/used/spec/Maker=10/Model=30735/Prefecture=1,2,9/Page={}", #LS
    "https://kakaku.com/kuruma/used/spec/Maker=10/Model=30735/Prefecture=17,24,29,36,42,47/Page={}",
    "https://kakaku.com/kuruma/used/spec/Maker=10/Model=30733/Page={}", #IS
    "https://kakaku.com/kuruma/used/spec/Maker=10/Model=33278/Page={}", #LC
    "https://kakaku.com/kuruma/used/spec/Maker=10/Model=33407/Page={}", #LCコンバーチブル
    "https://kakaku.com/kuruma/used/spec/Maker=10/Model=33132/Page={}", #RC
    "https://kakaku.com/kuruma/used/spec/Maker=10/Model=33134/Page={}", #RCF
    "https://kakaku.com/kuruma/used/spec/Maker=10/Model=30728/Page={}", #ES
    
]
 
for url in url_list:
    # 1~60ページまでスクレイピング
    for page in range(1, 61):
                 
        # 一覧ページのhtml取得
        soup = get_html(url.format(page))
        
        # 各ページの詳細情報を取得
        for item in soup.findAll("div", {"class": "ucItemBox"}):
            # url取得
            car_url = "https://kakaku.com" + item.find("a").get("href")
                         
            # 車情報取得
            all_data.append(parse_car_detail_info(car_url))
            
            # 1秒待つ
            sleep(1)
 
        # 次ページが存在するか確認
        if check_next_page(soup) == 0:
            break
 
 # Dataframeに変換
df = pd.DataFrame(all_data)
df.to_pickle("./Individuality.pkl")

hide_toggle()

取得したデータセットを簡略化し、表にしたものがこちらです。  
モデルの説明変数として使用するカラムを英語で示しています。

In [44]:
hide_toggle(for_next=True)

In [6]:
df = pd.read_pickle("./Individuality.pkl")

# カラムが100文字以上のものは排除
remove_columns = []
for i in df.columns:
    if len(i) >= 100:
        remove_columns.append(i)

df = df.drop(remove_columns, axis=1)

# 年式／初度登録：年式（西暦）を抽出
df["年式／初度登録"] = df["年式／初度登録"].fillna(df["年式／初度検査"])

df["Model_year"] = df["年式／初度登録"].apply(lambda x: int(re.findall(r"[0-9]{4}", x)[0]) if len(re.findall(r"[0-9]{4}", x))!=0 else np.nan)
df["Model_year"] = df["Model_year"].astype(float)

# 価格：無効データをNaNとして数値データに変換
df["Seling_price"] = df["価格"].replace("応談", np.nan)
df["Seling_price"] = df["Seling_price"].astype(float)


def convert_distance(value):
    if len(re.findall(r"[0-9.]+", value)) == 0:
        return 0
    else:      
        num = float(re.findall(r"[0-9.]+", value)[0])
           
        if "万" in value:
            num = float(num) * 10000

    return num


# モデル名を抽出
df["Model"] = df["モデル"].apply(lambda x: str(re.findall(r"(?<=「).+(?=」)", x)[0]) if len(re.findall(r"(?<=「).+(?=」)", x))!=0 else np.nan)

#メーカー
df.loc[df["メーカー"].str.startswith("トヨタ"), "Maker"] = 1
df.loc[df["メーカー"].str.startswith("ホンダ"), "Maker"] = 2
df.loc[df["メーカー"].str.startswith("日産"), "Maker"] = 3
df.loc[df["メーカー"].str.startswith("三菱"), "Maker"] = 4
df.loc[df["メーカー"].str.startswith("マツダ"), "Maker"] = 5
df.loc[df["メーカー"].str.startswith("スバル"), "Maker"] = 6
df.loc[df["メーカー"].str.startswith("レクサス"), "Maker"] = 7

# 価格あるいは年式がないものは排除
df.dropna(subset=["Seling_price", "Model_year"], how="any", inplace=True)

df

,名称,価格,色,年式／初度登録,走行距離,地域,車検,修復歴,未使用車,ワンオーナー,禁煙車,保証,ミッション,ハンドル,全長,駆動方式,車台番号,全幅,排気量,記録簿,全高,ドア数,リサイクル料,車両重量,乗車定員,整備,燃費（JC08モード）,燃料,使用燃料,モデル,メーカー,評価・鑑定,年式／初度検査,Model_year,Seling_price,Model,Maker
0,カムリ ＷＳレザーパッケージ 登録済未使用車 ナビ バックカメラ 黒革,409.8,パールホワイト,令和3年(2021年),30km,京都府,2024/02,修復歴なし,○,−,○,"販売店保証付(無料保証 期限:2026年2月 距離:100,000km ※いずれかの条件に達...",CVT,右ハンドル,"4,910mm※1",4WD,519（下3桁）,"1,840mm※1",2500cc,記録簿あり,"1,455mm※1",4ドア,リ済別,"1,680kg※1",5名,法定整備付,24.6km/L※1,ハイブリッド,レギュラー※1,※1 「カムリ 2017年モデル WS レザーパッケージ E-Four」のカタログによる情報...,トヨタ カムリ ＷＳレザーパッケージ 登録済未使用車 ナビ バックカメラ 黒革 409.8万...,NaN,NaN,2021.0,409.8,カムリ 2017年モデル WS レザーパッケージ E-Four,1.0
1,カムリ ＷＳ ハイブリッド 衝突被害軽減システム オートクルーズコントロール ＬＥＤヘッドラ...,345.0,プラチナホワイトパールマイカ,令和3年(2021年),931km,静岡県,2024/04,修復歴なし,−,○,−,販売店保証付(無料保証 期間:12ヵ月 距離:無制限 ※いずれかの条件に達するまで保証を適用...,AT,右ハンドル,"4,910mm※1",2WD,432（下3桁）,"1,840mm※1",2500cc,,"1,445mm※1",4ドア,リ済別,"1,570kg※1",5名,法定整備付,28.4km/L※1,ハイブリッド,レギュラー※1,※1 「カムリ 2017年モデル WS」のカタログによる情報です。実際の物件状態を保証するも...,トヨタ カムリ ＷＳ ハイブリッド 衝突被害軽減システム オートクルーズコントロール ＬＥＤ...,,NaN,2021.0,345.0,カムリ 2017年モデル WS,1.0
2,カムリ Ｘ メモリーナビ バックカメラ 衝突被害軽減システム ＥＴＣ ＬＥＤヘッドランプ ワ...,328.9,シルバーＭ,令和3年(2021年),997km,山口県,2024/01,修復歴なし,−,○,−,販売店保証付(無料保証 期間:12ヵ月 距離:無制限 ※いずれかの条件に達するまで保証を適用...,AT,右ハンドル,"4,885mm※1",2WD,601（下3桁）,"1,840mm※1",2500cc,,"1,445mm※1",4ドア,リ済別,"1,540kg※1",5名,法定整備付,33.4km/L※1,ハイブリッド,レギュラー※1,※1 「カムリ 2017年モデル X」のカタログによる情報です。実際の物件状態を保証するもの...,トヨタ カムリ Ｘ メモリーナビ バックカメラ 衝突被害軽減システム ＥＴＣ ＬＥＤヘッドラ...,,NaN,2021.0,328.9,カムリ 2017年モデル X,1.0
3,カムリ ＷＳ ワンオーナー 純正ナビ ドラレコ バックカメラ ヘットアップディスプレイ パノ...,377.0,アティチュードブラックマイカ,令和3年(2021年),0.4万km,北海道,2024/03,修復歴なし,−,○,○,"販売店保証付(無料保証 期限:2025年3月 距離:100,000km ※いずれかの条件に達...",CVT,右ハンドル,"4,910mm※1",4WD,706（下3桁）,"1,840mm※1",2500cc,記録簿あり,"1,455mm※1",4ドア,リ済別,"1,660kg※1",5名,法定整備付,24.6km/L※1,ハイブリッド,レギュラー※1,※1 「カムリ 2017年モデル WS E-Four」のカタログによる情報です。実際の物件状...,トヨタ カムリ ＷＳ ワンオーナー 純正ナビ ドラレコ バックカメラ ヘットアップディスプレ...,NaN,NaN,2021.0,377.0,カムリ 2017年モデル WS E-Four,1.0
4,カムリ Ｇレザーパッケージ ディーラー保証・走行距離無制限 社用車 サポカーＳ クルーズコン...,360.0,プラチナホワイトパールマイカ,令和3年(2021年),0.5万km,兵庫県,2024/03,修復歴なし,−,○,−,販売店保証付(無料保証 期間:12ヵ月 距離:無制限 ※いずれかの条件に達するまで保証を適用...,CVT,右ハンドル,"4,885mm※1",2WD,211（下3桁）,"1,840mm※1",2500cc,,"1,445mm※1",4ドア,リ済別,"1,600kg※1",5名,法定整備付,28.4km/L※1,ハイブリッド,レギュラー※1,※1 「カムリ 2017年モデル G レザーパッケージ」のカタログによる情報です。実際の物件...,トヨタ カムリ Ｇレザーパッケージ ディーラー保証・走行距離無制限 社用車 サポカーＳ クル...,,NaN,2021.0,360.0,カムリ 2017年モデル G レザーパッケージ,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27491,ＥＳ３００ｈ ３００ｈ バージョンＬ ブラック,548.0,クロ,令和元年(2019年),2.7万km,福岡県,車検整備なし,修復歴なし,−,−,−,販売店保証なし,AT,右ハンドル,"4,975mm※1",2WD,622（下3桁）,"1,865mm※1",2500cc,,"1,455mm※1",4ドア,,"1,730kg※1",5名,法定整備なし,23.4km/L※1,ハイブリッド,レギュラー※1,※1 「ES 2018年モデル 300h Version L」のカタログによる情報です。実際...,レクサス ＥＳ３００ｈ ３００ｈ バージョンＬ ブラック 548.0万円 令和元年(201...,,NaN,2019.0,548.0,ES 2018年モデル 300h Version L,7.0
27492,ＥＳ３００ｈ ３００ｈ Ｆスポーツ パール,508.0,パールホワイト,令和元年(2019年),2.8万km,福岡県,2022/12,修復歴なし,−,−,−,販売店保証なし,AT,右ハンドル,"4,975mm※1",2WD,070（下3桁）,"1,865mm※1",2500cc,,"1,455mm※1",4ドア,,"1,720kg※1",5名,法定整備なし,23.4km/L※1,ハイブリッド,レギュラー※1,※1 「ES 2018年モデル 300h F SPORT」のカタログによる情報です。実際の物...,レクサス ＥＳ３００ｈ ３００ｈ Ｆスポーツ パール 508.0万円 令和元年(2019年...,,NaN,2019.0,508.0,ES 2018年モデル 300h F SPORT,7.0
27493,ＥＳ３００ｈ バージョンＬ,479.0,パールホワイト,令和元年(2019年),4.9万km,埼玉県,車検整備なし,修復歴なし,−,−,−,販売店保証なし,AT,右ハンドル,"4,975mm※1",2WD,175（下3桁）,"1,865mm※1",2500cc,,"1,455mm※1",4ドア,リ済別,"1,730kg※1",5名,法定整備なし,23.4km/L※1,ハイブリッド,レギュラー※1,※1 「ES 2018年モデル 300h Version L」のカタログによる情報です。実際...,レクサス ＥＳ３００ｈ バージョンＬ 479.0万円 令和元年(2019年) 埼玉県 中古車...,NaN,NaN,2019.0,479.0,ES 2018年モデル 300h Version L,7.0
27494,ＥＳ３００ｈ ３００ｈ バージョンＬ サンルーフ／全方,529.8,パールホワイト,平成30年(2018年),2.2万km,福岡県,2023/12,修復歴なし,−,−,−,販売店保証なし,AT,右ハンドル,,2WD,781（下3桁）,,2500cc,,,4ドア,,,5名,法定整備なし,,ハイブリッド,,※ 掲載しているすべての情報は万全の保証をいたしかねます。,レクサス ＥＳ３００ｈ ３００ｈ バージョンＬ サンルーフ／全方 529.8万円 平成30年...,,NaN,2018.0,529.8,NaN,7.0


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27012 entries, 0 to 27495
Data columns (total 37 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   名称            27012 non-null  object 
 1   価格            27012 non-null  object 
 2   色             27012 non-null  object 
 3   年式／初度登録       27012 non-null  object 
 4   走行距離          27012 non-null  object 
 5   地域            27012 non-null  object 
 6   車検            27012 non-null  object 
 7   修復歴           27012 non-null  object 
 8   未使用車          27012 non-null  object 
 9   ワンオーナー        27012 non-null  object 
 10  禁煙車           27012 non-null  object 
 11  保証            27012 non-null  object 
 12  ミッション         27012 non-null  object 
 13  ハンドル          27012 non-null  object 
 14  全長            27012 non-null  object 
 15  駆動方式          27012 non-null  object 
 16  車台番号          27012 non-null  object 
 17  全幅            27012 non-null  object 
 18  排気量           27012 non-nu

  
今回は車の性能による価格変動を調査するため、  
個々の車両の使用状況や価格に影響を及ぼさないと予想される要素を削除します。

In [12]:
drop_columns = ["走行距離", "地域", "車検","修復歴", "未使用車", "ワンオーナー", "禁煙車","保証", "ハンドル", "車台番号", "記録簿", "リサイクル料", "整備", "燃費（JC08モード）", "評価・鑑定", "燃費（JC08モード）", "年式／初度検査"]
df = df.drop(drop_columns, axis=1)
df.info()
hide_toggle()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 27012 entries, 0 to 27495
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   名称            27012 non-null  object 
 1   価格            27012 non-null  object 
 2   色             27012 non-null  object 
 3   年式／初度登録       27012 non-null  object 
 4   ミッション         27012 non-null  object 
 5   全長            27012 non-null  object 
 6   駆動方式          27012 non-null  object 
 7   全幅            27012 non-null  object 
 8   排気量           27012 non-null  object 
 9   全高            27012 non-null  object 
 10  ドア数           27012 non-null  object 
 11  車両重量          27012 non-null  object 
 12  乗車定員          27012 non-null  object 
 13  燃料            27012 non-null  object 
 14  使用燃料          27012 non-null  object 
 15  モデル           27012 non-null  object 
 16  メーカー          27012 non-null  object 
 17  Model_year    27012 non-null  float64
 18  Seling_price  27012 non-nu

このデータセットには欠損値として扱うべき値"","-"が含まれています。  
欠損値の処理は最後に行います。  
まずはその元となるデータ"Model"列を補正していきます。  
そのために必要な情報を数値化します。

In [16]:
df.loc[df["色"].str.contains("ブラック|クロ"), "Color"] = 0
df.loc[df["色"].str.contains("グレー"), "Color"] = 1
df.loc[df["色"].str.contains("シルバー"), "Color"] = 2
df.loc[df["色"].str.contains("ホワイト|シロ"), "Color"] = 3
df.loc[df["色"].str.contains("レッド|アカ"), "Color"] = 4
df.loc[df["色"].str.contains("ブルー|アオ"), "Color"] = 5
df.loc[df["色"]=="パール", "Color"] = 3

df.loc[df["色"].str.contains("パール|P"), "Brightness"] = 1 #パール
df.loc[df["色"].str.contains("メタリック|M"), "Brightness"] = 2 #メタリック
df.loc[df["Brightness"].isnull(), "Brightness"] = 0

df["MT"] = df["ミッション"].apply(lambda x: 1 if x=="MT" else 0)

df["WD"] = df["駆動方式"].apply(lambda x: int(re.findall(r"[0-9]", x)[0]) if len(re.findall(r"[0-9]", x))!=0 else np.nan)
df["WD"] = df["WD"].astype(float)

df["Displacement"] = df["排気量"].apply(lambda x: float(re.findall(r"[0-9]+", x)[0]) if len(re.findall(r"[0-9]+", x))!=0 else np.nan)

df["Door"] = df["ドア数"].apply(lambda x: int(re.findall(r"[0-9]", x)[0]) if len(re.findall(r"[0-9]", x))!=0 else np.nan)
df["Door"] = df["Door"].astype(float)

df["Capacity"] = df["乗車定員"].apply(lambda x: int(re.findall(r"[0-9]", x)[0]) if len(re.findall(r"[0-9]", x))!=0 else np.nan)
df["Capacity"] = df["Capacity"].astype(float)

df["Length"] = df["全長"].apply(lambda x: str(re.findall(r"[0-9,]+", x)[0]) if len(re.findall(r"[0-9,]+", x))!=0 else np.nan)
df["Length"] = df["Length"].str.replace(",","")
df["Length"] = df["Length"].astype(float)

df["Width"] = df["全幅"].apply(lambda x: str(re.findall(r"[0-9,]+", x)[0]) if len(re.findall(r"[0-9,]+", x))!=0 else np.nan)
df["Width"] = df["Width"].str.replace(",","")
df["Width"] = df["Width"].astype(float)

df["Height"] = df["全高"].apply(lambda x: str(re.findall(r"[0-9,]+", x)[0]) if len(re.findall(r"[0-9,]+", x))!=0 else np.nan)
df["Height"] = df["Height"].str.replace(",","")
df["Height"] = df["Height"].astype(float)

df["Weight"] = df["車両重量"].apply(lambda x: str(re.findall(r"[0-9,]+", x)[0]) if len(re.findall(r"[0-9,]+", x))!=0 else np.nan)
df["Weight"] = df["Weight"].str.replace(",","")
df["Weight"] = df["Weight"].astype(float)

df.loc[df["名称"].str.contains("台限定"), "Limited"] = 1
df.loc[~df["名称"].str.contains("台限定"), "Limited"] = 0

df["Age"] = 2022-df["Model_year"]

hide_toggle()

個々の中古車両のタイトルや性能、年式などをもとに、  
"Model"列を一つ一つ補正していきます。

In [66]:
# トヨタ

df.loc[(df["Model"].isnull())&(df["名称"].str.contains("カムリ"))&(df["Model_year"]>2016)&(df["WD"]==4), "Model"] = "カムリ 2017年モデル WS E-Four"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("カムリ"))&(df["Model_year"]>2016)&(df["名称"].str.contains("Ｇ|G")), "Model"] = "カムリ 2017年モデル G"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("カムリ"))&(df["Model_year"]>2016)&(df["名称"].str.contains("ＷＳ|WS")), "Model"] = "カムリ 2017年モデル WS"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("カムリ"))&(df["Model_year"]>2016)&(df["名称"].str.contains("Ｘ|X")), "Model"] = "カムリ 2017年モデル X"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("カムリ"))&(df["Model_year"]>2016), "Model"] = "カムリ 2017年モデル G"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("カムリ"))&(df["Model_year"]>2010)&(df["Model_year"]<2017)&(df["名称"].str.contains("Ｇ|G")), "Model"] = "カムリ 2011年モデル ハイブリッド Gパッケージ"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("カムリ"))&(df["Model_year"]>2010)&(df["Model_year"]<2017), "Model"] = "カムリ 2011年モデル ハイブリッド レザーパッケージ"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("カムリ"))&(df["Model_year"]>2010)&(df["Model_year"]<2017), "Model"] = "カムリ 2011年モデル ハイブリッド"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("カムリ"))&(df["名称"].str.contains("G")), "Model"] = "カムリ 2017年モデル G E-Four"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("カムリ")), "Model"] = "カムリ 2017年モデル G"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("マークＸ|マークX"))&(df["Model_year"]>2008)&(df["WD"]==4), "Model"] = "マークX 2009年モデル 250G Four"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("マークＸ|マークX"))&(df["Model_year"]>2008)&(df["名称"].str.contains("３５０Ｓ　Ｇ’ｓ|３５０Ｓ　Ｇｓ")), "Model"] = "マークX 2009年モデル 350S G's"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("マークＸ|マークX"))&(df["Model_year"]>2008)&(df["Displacement"]>3400), "Model"] = "マークX 2009年モデル 350S"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("マークＸ|マークX"))&(df["Model_year"]>2008)&(df["Displacement"]>2400), "Model"] = "マークX 2009年モデル 250G"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("マークＸ|マークX"))&(df["Model_year"]<2009)&(df["Displacement"]>2400), "Model"] = "マークX 2004年モデル 250G"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("スープラ"))&(df["Model_year"]>2018)&(df["Displacement"]>2800), "Model"] = "スープラ 2019年モデル RZ"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("スープラ"))&(df["Model_year"]>2018)&(df["Displacement"]==2000), "Model"] = "スープラ 2019年モデル SZ-R"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("スープラ"))&(df["Model_year"]<2019)&(df["Displacement"]==3000)&(df["名称"].str.contains("Ｒ")), "Model"] = "スープラ 1993年モデル RZ-S"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("スープラ"))&(df["Model_year"]<2019)&(df["Displacement"]==3000)&(~df["名称"].str.contains("Ｒ")), "Model"] = "スープラ 1993年モデル SZ"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("スープラ"))&(df["Model_year"]<2019)&(df["Displacement"]<2100), "Model"] = "スープラ 1993年モデル 2.0GT"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("スープラ"))&(df["Model_year"]<2019)&(df["Displacement"]>2400), "Model"] = "スープラ 1993年モデル 2.5GT"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("カローラ"))&(df["Displacement"]>1700)&(df["名称"].str.contains("Ｚ|Z")), "Model"] = "カローラ スポーツ 2018年モデル HYBRID G Z"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("カローラ"))&(df["Displacement"]>1700)&(df["名称"].str.contains("X")), "Model"] = "カローラ スポーツ 2018年モデル HYBRID G X"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("カローラ"))&(df["Displacement"]>1700), "Model"] = "カローラ スポーツ 2018年モデル HYBRID G"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("カローラ"))&(df["Displacement"]==1200)&(df["WD"]==4), "Model"] = "カローラ スポーツ 2018年モデル G Z 4WD"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("カローラ"))&(df["Displacement"]==1200)&(df["WD"]==2)&(df["名称"].str.contains("Ｚ|Z")), "Model"] = "カローラ スポーツ 2018年モデル G Z"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("カローラ"))&(df["Displacement"]==1200)&(df["WD"]==2)&(~df["名称"].str.contains("Ｚ|Z")), "Model"] = "カローラ スポーツ 2018年モデル G"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("カローラ")), "Model"] = "カローラ スポーツ 2018年モデル G Z"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("クラウン"))&(df["Model_year"]>2011)&(df["WD"]==4)&(df["名称"].str.contains("S"))&(df["名称"].str.contains("i")), "Model"] = "クラウン アスリート 2012年モデル 2.5 アスリートS i-Four"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("クラウン"))&(df["Model_year"]>2011)&(df["WD"]==4)&(df["名称"].str.contains("S"))&(~df["名称"].str.contains("i")), "Model"] = "クラウン アスリート 2012年モデル 2.5 アスリートS Four"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("クラウン"))&(df["Model_year"]>2011)&(df["WD"]==4)&(df["名称"].str.contains("G"))&(df["名称"].str.contains("i")), "Model"] = "クラウン アスリート 2012年モデル 2.5 アスリートG i-Four"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("クラウン"))&(df["Model_year"]>2011)&(df["WD"]==4)&(df["名称"].str.contains("i")), "Model"] = "クラウン アスリート 2012年モデル 2.5 アスリート i-Four"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("クラウン"))&(df["Model_year"]>2011)&(df["WD"]==4)&(~df["名称"].str.contains("i")), "Model"] = "クラウン アスリート 2012年モデル 2.5 アスリート Four"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("クラウン"))&(df["Model_year"]>2011)&(df["WD"]==2)&(df["名称"].str.contains("S"))&(df["Displacement"]>3400), "Model"] = "クラウン アスリート 2012年モデル 3.5 アスリートS"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("クラウン"))&(df["Model_year"]>2011)&(df["WD"]==2)&(df["名称"].str.contains("S"))&(df["Displacement"]>2400), "Model"] = "クラウン アスリート 2012年モデル 2.5 アスリートS"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("クラウン"))&(df["Model_year"]>2011)&(df["WD"]==2)&(df["名称"].str.contains("G"))&(df["Displacement"]>2400), "Model"] = "クラウン アスリート 2012年モデル 2.5 アスリートG"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("クラウン"))&(df["Model_year"]>2011)&(df["WD"]==2)&(df["名称"].str.contains("リミテッド"))&(df["Displacement"]>2400), "Model"] = "クラウン アスリート 2012年モデル ハイブリッド アスリートS J-FRONTIER Limited"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("クラウン"))&(df["Model_year"]>2011)&(df["WD"]==2)&(df["Displacement"]>2400), "Model"] = "クラウン アスリート 2012年モデル 2.5 アスリート"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("クラウン"))&(df["Model_year"]>2011)&(df["WD"]==2)&(df["Displacement"]<2100)&(df["名称"].str.contains("フロンティア")), "Model"] = "クラウン アスリート 2012年モデル S-T J-FRONTIER"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("クラウン"))&(df["Model_year"]>2011)&(df["WD"]==2)&(df["Displacement"]<2100)&(~df["名称"].str.contains("フロンティア")), "Model"] = "クラウン アスリート 2012年モデル S-T"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("クラウン"))&(df["WD"]==4)&(df["名称"].str.contains("60th")), "Model"] = "クラウン アスリート 2003年モデル 2.5 アスリートi-Four 60th スペシャルエディション"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("クラウン"))&(df["WD"]==4)&(df["名称"].str.contains("ナビパッケージ")), "Model"] = "クラウン アスリート 2008年モデル 2.5 アスリート i-Four ナビパッケージ"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("クラウン"))&(df["WD"]==4), "Model"] = "クラウン アスリート 2008年モデル 2.5 アスリート i-Four"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("クラウン"))&(df["Model_year"]<2012)&(df["Model_year"]>2002)&(df["WD"]==2)&(df["Displacement"]>3400), "Model"] = "クラウン アスリート 2008年モデル 3.5 アスリート"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("クラウン"))&(df["Model_year"]<2012)&(df["Model_year"]>2002)&(df["WD"]==2)&(df["Displacement"]>2400), "Model"] = "クラウン アスリート 2008年モデル 2.5 アスリート"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("クラウン"))&(df["WD"]==2)&(df["Displacement"]==3000), "Model"] = "クラウン アスリート 2003年モデル 3.0 アスリート"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("クラウン"))&(df["Displacement"]==3500), "Model"] = "クラウン アスリート 2008年モデル 3.5 アスリート"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("クラウン"))&(df["名称"].str.contains("アニバーサリー")), "Model"] = "クラウン アスリート 2008年モデル 2.5 アスリート Anniversary Edition"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("クラウン"))&(df["名称"].str.contains("フロンティア")), "Model"]  = "クラウン アスリート 2012年モデル S-T J-FRONTIER"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("クラウン"))&(df["名称"].str.contains("プレミアム")), "Model"]  = "クラウン アスリート 2008年モデル 2.5 アスリート Premium Edition"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("クラウン")), "Model"] = "クラウン アスリート 2008年モデル 2.5 アスリート"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("86|８６"))&(df["Displacement"]>1900)&(df["Displacement"]<2100)&(df["名称"].str.contains("リミテッド")), "Model"]  = "86 2012年モデル GT Limited"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("86|８６"))&(df["Displacement"]>1900)&(df["Displacement"]<2100)&(df["名称"].str.contains("ＧＴ|GT")), "Model"]  = "86 2012年モデル GT"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("86|８６"))&(df["Displacement"]>1900)&(df["Displacement"]<2100)&(df["名称"].str.contains("Ｃｂ")), "Model"]  = "86 2012年モデル style Cb"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("86|８６"))&(df["Displacement"]>1900)&(df["Displacement"]<2100)&(df["名称"].str.contains("１４Ｒ|14R")), "Model"]  = "86 2012年モデル 14R"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("86|８６"))&(df["Displacement"]>1900)&(df["Displacement"]<2100)&(df["名称"].str.contains("Ｇ|G")), "Model"]  = "86 2012年モデル G"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("86|８６"))&(df["Displacement"]>1900)&(df["Displacement"]<2100), "Model"]  = "86 2012年モデル GT"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ヤリス"))&(df["名称"].str.contains("ハイパフォーマンス|ハイ　パフォーマンス")), "Model"]  = "GRヤリス 2020年モデル RZ High performance"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ヤリス"))&(df["名称"].str.contains("RZ")), "Model"]  = "GRヤリス 2020年モデル RZ"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ヤリス"))&(df["名称"].str.contains("RC")), "Model"]  = "GRヤリス 2020年モデル RC"

# ホンダ

df.loc[(df["Model"].isnull())&(df["名称"].str.contains("シビック"))&(df["名称"].str.contains("タイプＲ|タイプR"))&(df["Model_year"]>2016), "Model"]  = "シビック タイプR 2017年モデル ベースグレード"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("シビック"))&(df["名称"].str.contains("タイプＲ|タイプR"))&(df["Model_year"]>2014), "Model"]  = "シビック タイプR 2015年モデル ベースグレード"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("シビック"))&(df["名称"].str.contains("タイプＲ|タイプR|ＴＹＰＥ Ｒ"))&(df["名称"].str.contains("ユーロ|ＥＵＲＯ"))&(df["Model_year"]>2006), "Model"]  = "シビック タイプR 2007年モデル タイプR ユーロ (MT)"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("シビック"))&(df["名称"].str.contains("タイプＲ|タイプR"))&(~df["名称"].str.contains("ユーロ"))&(df["Model_year"]>2006), "Model"]  = "シビック タイプR 2007年モデル タイプR (MT)"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("シビック"))&(df["名称"].str.contains("タイプＲ|タイプR"))&(df["Model_year"]>2000), "Model"]  = "シビック タイプR 2001年モデル タイプR (MT)"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("シビック"))&(df["名称"].str.contains("タイプＲ|タイプR"))&(df["Model_year"]<2001), "Model"]  = "シビック タイプR 1990年モデル タイプR"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("シビック"))&(df["名称"].str.contains("ＭＵＧＥＮ　ＲＲ")), "Model"]  = "シビック 2005年モデル MUGEN RR"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("シビック"))&(df["名称"].str.contains("ＳｉＲ|SiR")), "Model"]  = "シビック 1990年モデル SiR"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("シビック"))&(df["名称"].str.contains("セダン")), "Model"]  = "シビック セダン 2017年モデル ベースグレード"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("シビック"))&(df["名称"].str.contains("ＥＸ"))&(df["Model_year"]>2020), "Model"]  = "シビック 2021年モデル EX"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("シビック"))&(df["Model_year"]>2016)&(df["MT"]==1), "Model"]  = "シビック 2017年モデル ハッチバック (MT)"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("シビック"))&(df["Model_year"]>2016)&(df["MT"]==0), "Model"]  = "シビック 2017年モデル ハッチバック"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("シビック"))&(df["Model_year"]<2000), "Model"]  = "シビック 1990年モデル"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("シビック")), "Model"]  = "シビック 2005年モデル MX"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("Ｓ６６０|S660"))&(df["名称"].str.contains("モデューロ")), "Model"]  = "S660 2015年モデル Modulo X"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("Ｓ６６０|S660"))&(df["名称"].str.contains("ＭＵＧＥＮ　ＲＡ")), "Model"]  = "S660 2015年モデル MUGEN RA"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("Ｓ６６０|S660"))&(df["名称"].str.contains("トラッドレザーエディション")), "Model"]  = "S660 2015年モデル α Trad Leather Edition"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("Ｓ６６０|S660"))&(df["名称"].str.contains("β|ベータ"))&(df["MT"]==1), "Model"]  = "S660 2015年モデル β (MT)"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("Ｓ６６０|S660"))&(df["名称"].str.contains("β|ベータ"))&(df["MT"]==0), "Model"]  = "S660 2015年モデル β"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("Ｓ６６０|S660"))&(df["MT"]==1), "Model"]  = "S660 2015年モデル α (MT)"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("Ｓ６６０|S660"))&(df["MT"]==0), "Model"]  = "S660 2015年モデル α"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("アコード"))&(df["Model_year"]>2019), "Model"]  = "アコード 2020年モデル EX"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("アコード"))&(df["Model_year"]>2012), "Model"]  = "アコード 2008年モデル 24TL"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("アコード"))&(df["Model_year"]>2007)&(df["Displacement"]==2400), "Model"]  = "アコード 2008年モデル 24TL"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("アコード"))&(df["Model_year"]>2001)&(df["Displacement"]==2000), "Model"]  = "アコード 2002年モデル 20EL"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("アコード"))&(df["Model_year"]>2001)&(df["Displacement"]>2000), "Model"]  = "アコード 2002年モデル 24S"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("アコード")), "Model"]  = "アコード 1990年モデル"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＮＳＸ|NSX"))&(df["Model_year"]>2016), "Model"]  = "NSX 2017年モデル ベースグレード"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＮＳＸ|NSX"))&(df["Model_year"]<2017), "Model"]  = "NSX 1990年モデル ベースグレード"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("Ｓ２０００|S2000"))&(df["名称"].str.contains("タイプＳ")), "Model"]  = "S2000 1999年モデル S2000 タイプ S (MT)"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("Ｓ２０００|S2000"))&(df["名称"].str.contains("タイプＶ")), "Model"]  = "S2000 1999年モデル S2000 タイプV"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("Ｓ２０００|S2000")), "Model"]  = "S2000 1999年モデル S2000 (MT)"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("インテグラ"))&(df["名称"].str.contains("タイプＲ")), "Model"]  = "インテグラ 2001年モデル タイプ R (MT)"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("インテグラ"))&(df["名称"].str.contains("タイプＳ")), "Model"]  = "インテグラ 2001年モデル タイプ S (MT)"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("インテグラ")), "Model"]  = "インテグラ 2001年モデル iS (MT)"

# 日産

df.loc[(df["Model"].isnull())&(df["名称"].str.contains("スカイライン"))&(df["名称"].str.contains("３５０ＧＴ"))&(df["Model_year"]>2005), "Model"]  = "スカイライン 2014年モデル 350GT HYBRID Type SP"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("スカイライン"))&(df["Displacement"]==3700)&(df["Model_year"]>2005), "Model"]  = "スカイライン 2006年モデル 370GT タイプS"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("スカイライン"))&(df["Displacement"]>3400)&(df["Model_year"]>2000)&(df["MT"]==1), "Model"]  = "スカイライン 2001年モデル 350GT (MT)"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("スカイライン"))&(df["Displacement"]>3400)&(df["Model_year"]>2000)&(df["MT"]==0), "Model"]  = "スカイライン 2001年モデル 350GT"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("スカイライン"))&(df["Displacement"]>2900)&(df["Model_year"]>2000), "Model"]  = "スカイライン 2001年モデル 300GT"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("スカイライン"))&(df["Displacement"]>2400)&(df["Model_year"]>2000), "Model"]  = "スカイライン 2001年モデル 250GT"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("スカイライン"))&(df["Displacement"]>1900)&(df["Model_year"]>2000), "Model"]  = "スカイライン 2001年モデル 200GT"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("スカイライン"))&(~df["名称"].str.contains("フェアレディ"))&(df["Model_year"]<2001), "Model"]  = "スカイライン 1970年モデル GT"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("フーガ"))&(df["名称"].str.contains("ハイブリッド")), "Model"]  = "フーガ 2009年モデル ハイブリッド"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("フーガ"))&(df["Model_year"]>2008)&(df["名称"].str.contains("250VIP")), "Model"]  = "フーガ 2009年モデル 250VIP"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("フーガ"))&(df["Model_year"]>2008)&(df["名称"].str.contains("250GT|２５０ＧＴ")), "Model"]  = "フーガ 2009年モデル 250GT"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("フーガ"))&(df["Model_year"]>2008)&(df["名称"].str.contains("タイプS|タイプS")), "Model"]  = "フーガ 2009年モデル 370GT Type S"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("フーガ"))&(df["Model_year"]>2008)&(df["WD"]==4), "Model"]  = "フーガ 2009年モデル 370GT FOUR"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("フーガ"))&(df["Model_year"]>2008)&(df["Displacement"]>3600), "Model"]  = "フーガ 2009年モデル 370GT"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("フーガ"))&(df["Model_year"]<2009)&(df["名称"].str.contains("タイプS|タイプS")), "Model"]  = "フーガ 2004年モデル 350GT TypeS"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("フーガ"))&(df["Model_year"]<2009)&(df["名称"].str.contains("タイプP|タイプP")), "Model"]  = "フーガ 2004年モデル 350GT TypeP"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("フーガ"))&(df["Model_year"]<2009)&(df["Displacement"]>3400)&(df["WD"]==4), "Model"]  = "フーガ 2004年モデル 350GT FOUR"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("フーガ"))&(df["Model_year"]<2009)&(df["Displacement"]>3400)&(~(df["WD"]==4)), "Model"]  = "フーガ 2004年モデル 350GT"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("フーガ"))&(df["Model_year"]<2009), "Model"]  = "フーガ 2004年モデル 250GT"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("フェアレディ"))&(df["Model_year"]>2007)&(df["名称"].str.contains("ＮＩＳＭＯ"))&(df["MT"]==1), "Model"]  = "フェアレディZ 2008年モデル Version NISMO (MT)"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("フェアレディ"))&(df["Model_year"]>2007)&(df["名称"].str.contains("ＮＩＳＭＯ"))&(df["MT"]==0), "Model"]  = "フェアレディZ 2008年モデル Version NISMO"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("フェアレディ"))&(df["Model_year"]>2007)&(df["MT"]==1), "Model"]  = "フェアレディZ 2008年モデル ベースグレード (MT)"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("フェアレディ"))&(df["Model_year"]>2007)&(df["MT"]==0), "Model"]  = "フェアレディZ 2008年モデル ベースグレード"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("フェアレディ"))&(df["Model_year"]>2001)&(df["名称"].str.contains("ニスモ"))&(df["MT"]==1), "Model"]  = "フェアレディZ 2002年モデル Version NISMO (MT)"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("フェアレディ"))&(df["Model_year"]>2001)&(df["名称"].str.contains("ニスモ"))&(df["MT"]==0), "Model"]  = "フェアレディZ 2002年モデル Version NISMO"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("フェアレディ"))&(df["Model_year"]>2001)&(df["名称"].str.contains("ロードスター")), "Model"]  = "フェアレディZ 2002年モデル ロードスター"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("フェアレディ"))&(df["Model_year"]>2001)&(df["MT"]==1), "Model"]  = "フェアレディZ 2002年モデル ベースグレード (MT)"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("フェアレディ"))&(df["Model_year"]>2001)&(df["MT"]==0), "Model"]  = "フェアレディZ 2002年モデル ベースグレード"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("フェアレディ"))&(df["Model_year"]>1988), "Model"]  = "フェアレディZ 1989年モデル ベースグレード"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("フェアレディ"))&(df["Model_year"]>1977), "Model"]  = "フェアレディZ 1978年モデル ベースグレード"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("フェアレディ"))&(df["Model_year"]<1978), "Model"]  = "フェアレディZ 1969年モデル ベースグレード"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＧＴ−Ｒ")), "Model"]  = "GT-R 2007年モデル GT-R"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("GT-R"))&(df["名称"].str.contains("ブラック")), "Model"]  = "GT-R 2007年モデル GT-R Black edition"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("GT-R"))&(df["名称"].str.contains("ピュア")), "Model"]  = "GT-R 2007年モデル GT-R Pure edition"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("GT-R"))&(df["名称"].str.contains("プレミアム")), "Model"]  = "GT-R 2007年モデル GT-R Premium edition"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("GT-R")), "Model"]  = "GT-R 2007年モデル GT-R"

# 三菱

df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ランサー"))&(df["Model_year"]>2006), "Model"]  = "ランサーエボリューション 2007年モデル X GSR (MT)"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ランサー"))&(df["Model_year"]<2007), "Model"]  = "ランサーエボリューション 2001年モデル IX GSR 4WD (MT)"

# マツダ

df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＭＡＺＤＡ３"))&(df["WD"]==4), "Model"]  = "MAZDA3 ファストバック 2019年モデル 15S Touring 4WD"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＭＡＺＤＡ３"))&(df["MT"]==1), "Model"]  = "MAZDA3 ファストバック 2019年モデル 15S Touring (MT)"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＭＡＺＤＡ３|MAZDA3"))&(df["Displacement"]>1900)&(df["名称"].str.contains("バーガンディ")), "Model"]  = "MAZDA3 ファストバック 2019年モデル 20S Burgundy Selection"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＭＡＺＤＡ３|MAZDA3"))&(df["Displacement"]>1900)&(df["名称"].str.contains("プロアクティブ")), "Model"]  = "MAZDA3 ファストバック 2019年モデル 20S PROACTIVE"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＭＡＺＤＡ３|MAZDA3"))&(df["Displacement"]>1900), "Model"]  = "MAZDA3 ファストバック 2019年モデル 20S L Package"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＭＡＺＤＡ３|MAZDA3"))&(df["Displacement"]>1700), "Model"]  = "MAZDA3 ファストバック 2019年モデル XD L Package"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＭＡＺＤＡ３|MAZDA3"))&(df["Displacement"]>1400), "Model"]  = "MAZDA3 ファストバック 2019年モデル 15S Touring"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("アクセラスポーツ"))&(df["Model_year"]>2012)&(df["Displacement"]>2100)&(df["名称"].str.contains("プロアクティブ")), "Model"]  = "アクセラスポーツ 2013年モデル 22XD PROACTIVE"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("アクセラスポーツ"))&(df["Model_year"]>2012)&(df["Displacement"]>2100), "Model"]  = "アクセラスポーツ 2013年モデル 22XD L Package"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("アクセラスポーツ"))&(df["Model_year"]>2012)&(df["Displacement"]>1900)&(df["名称"].str.contains("プロアクティブ")), "Model"]  = "アクセラスポーツ 2013年モデル 20S Touring"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("アクセラスポーツ"))&(df["Model_year"]>2012)&(df["Displacement"]>1400)&(df["名称"].str.contains("プロアクティブ")), "Model"]  = "アクセラスポーツ 2013年モデル 15S PROACTIVE"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("アクセラスポーツ"))&(df["Model_year"]>2012)&(df["Displacement"]>1400), "Model"]  = "アクセラスポーツ 2013年モデル 15S"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("アクセラスポーツ"))&(df["Model_year"]>2008)&(df["Displacement"]>1900), "Model"]  = "アクセラスポーツ 2009年モデル 20S"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("アクセラスポーツ"))&(df["Model_year"]>2008)&(df["Displacement"]>1400), "Model"]  = "アクセラスポーツ 2009年モデル 15C"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("アクセラスポーツ"))&(df["Model_year"]<2009)&(df["Displacement"]>1900), "Model"]  = "アクセラスポーツ 2003年モデル 20S"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("アクセラスポーツ"))&(df["Model_year"]<2009)&(df["Displacement"]>1400), "Model"]  = "アクセラスポーツ 2003年モデル 15C (MT)"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("アクセラスポーツ")), "Model"]  = "アクセラスポーツ 2013年モデル 15S"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("アクセラ"))&(df["Model_year"]>2012)&(df["Displacement"]>2100)&(df["名称"].str.contains("プロアクティブ"))&(df["WD"]==4), "Model"]  = "アクセラセダン 2013年モデル 22XD PROACTIVE 4WD"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("アクセラ"))&(df["Model_year"]>2012)&(df["Displacement"]>2100)&(df["名称"].str.contains("プロアクティブ")), "Model"]  = "アクセラセダン 2013年モデル 22XD PROACTIVE"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("アクセラ"))&(df["Model_year"]>2012)&(df["Displacement"]>2100), "Model"]  = "アクセラセダン 2013年モデル 22XD L Package"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("アクセラ"))&(df["Model_year"]>2012)&(df["Displacement"]>1900)&(df["名称"].str.contains("プロアクティブ")), "Model"]  = "アクセラセダン 2013年モデル HYBRID-S PROACTIVE"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("アクセラ"))&(df["Model_year"]>2012)&(df["Displacement"]>1900), "Model"]  = "アクセラセダン 2013年モデル HYBRID-S L Package"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("アクセラ"))&(df["Model_year"]>2012)&(df["Displacement"]>1400)&(df["名称"].str.contains("プロアクティブ"))&(df["WD"]==4), "Model"]  = "アクセラセダン 2013年モデル 15S PROACTIVE 4WD"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("アクセラ"))&(df["Model_year"]>2012)&(df["Displacement"]>1400)&(df["名称"].str.contains("プロアクティブ")), "Model"]  = "アクセラセダン 2013年モデル 15S PROACTIVE"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("アクセラ"))&(df["Model_year"]>2012)&(df["Displacement"]>1400), "Model"]  = "アクセラセダン 2013年モデル 15S"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("アクセラ"))&(df["Model_year"]<2013), "Model"]  = "アクセラセダン 2009年モデル 15C CVT"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ロードスターRF|ロードスターRF"))&(df["名称"].str.contains("RS")), "Model"]  = "ロードスター RF 2016年モデル RS"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ロードスターRF|ロードスターRF"))&(df["名称"].str.contains("VS"))&(df["MT"]==1), "Model"]  = "ロードスター RF 2016年モデル VS (MT)"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ロードスターRF|ロードスターRF"))&(df["名称"].str.contains("VS")), "Model"]  = "ロードスター RF 2016年モデル VS"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ロードスターRF|ロードスターRF")), "Model"]  = "ロードスター RF 2016年モデル 30周年記念車 (MT)"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ロードスターＲＦ")), "Model"]  = "ロードスター RF 2016年モデル S (MT)"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ロードスター"))&(df["Model_year"]>2014)&(df["MT"]==1), "Model"]  = "ロードスター 2015年モデル S Special Package (MT)"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ロードスター"))&(df["Model_year"]>2014), "Model"]  = "ロードスター 2015年モデル S Special Package"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ロードスター"))&(df["Model_year"]>2004)&(df["名称"].str.contains("ＲＨＴ"))&(df["名称"].str.contains("VS"))&(df["MT"]==1), "Model"]  = "ロードスター 2005年モデル VS RHT (MT)"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ロードスター"))&(df["Model_year"]>2004)&(df["名称"].str.contains("ＲＨＴ"))&(df["名称"].str.contains("VS")), "Model"]  = "ロードスター 2005年モデル VS RHT"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ロードスター"))&(df["Model_year"]>2004)&(df["名称"].str.contains("ＲＨＴ"))&(df["名称"].str.contains("ＲＳ")), "Model"]  = "ロードスター 2005年モデル RS RHT"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ロードスター"))&(df["Model_year"]>2004)&(df["名称"].str.contains("ＲＨＴ"))&(df["名称"].str.contains("Ｓ")), "Model"]  = "ロードスター 2005年モデル S RHT"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ロードスター"))&(df["Model_year"]>2004)&(df["名称"].str.contains("ＲＨＴ")), "Model"]  = "ロードスター 2005年モデル RHT (MT)"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ロードスター"))&(df["Model_year"]>2004)&(df["名称"].str.contains("VS"))&(df["MT"]==1), "Model"]  = "ロードスター 2005年モデル VS (MT)"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ロードスター"))&(df["Model_year"]>2004)&(df["名称"].str.contains("VS")), "Model"]  = "ロードスター 2005年モデル VS"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ロードスター"))&(df["Model_year"]>2004)&(df["名称"].str.contains("ＲＳ")), "Model"]  = "ロードスター 2005年モデル RS"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ロードスター"))&(df["Model_year"]>2004)&(df["名称"].str.contains("Ｓ")), "Model"]  = "ロードスター 2005年モデル S"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ロードスター"))&(df["Model_year"]>2003)&(df["名称"].str.contains("ＮＲ−Ａ")), "Model"]  = "ロードスター 2005年モデル NR-A (MT)"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ロードスター"))&(df["Model_year"]>2004), "Model"]  = "ロードスター 2005年モデル ロードスター"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ロードスター"))&(df["Model_year"]>1997)&(df["名称"].str.contains("ターボ")), "Model"]  = "ロードスター 1998年モデル ロードスター ターボ"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ロードスター"))&(df["Model_year"]>1997), "Model"]  = "ロードスター 1998年モデル"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ロードスター"))&(df["Model_year"]<1998), "Model"]  = "ロードスター 1989年モデル"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＲＸ−７|RX-7"))&(df["名称"].str.contains("ＲＳ|RS")), "Model"]  = "RX-7 1991年モデル タイプ RS (MT)"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＲＸ−７|RX-7")), "Model"]  = "RX-7 1991年モデル タイプ RB (MT)"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＲＸ−８|RX-8"))&(df["名称"].str.contains("タイプR")), "Model"]  = "RX-8 2003年モデル Type RS"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＲＸ−８|RX-8"))&(df["名称"].str.contains("タイプS")), "Model"]  = "RX-8 2003年モデル Type S"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＲＸ−８|RX-8"))&(df["名称"].str.contains("タイプE")), "Model"]  = "RX-8 2003年モデル Type E"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＲＸ−８|RX-8"))&(df["名称"].str.contains("タイプG")), "Model"]  = "RX-8 2003年モデル Type G"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＲＸ−８|RX-8"))&(df["名称"].str.contains("スピリット")), "Model"]  = "RX-8 2003年モデル SPIRIT R"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＲＸ−８|RX-8")), "Model"]  = "RX-8 2003年モデル ベースグレード (MT)"

# スバル

df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＷＲＸ Ｓ４|ＷＲＸ　Ｓ４|WRX S4")), "Model"]  = "WRX S4 2014年モデル 2.0GT EyeSight"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("WRX 2.0GT-S")), "Model"]  = "WRX S4 2014年モデル 2.0GT-S EyeSight"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＷＲＸ ＳＴＩ|ＷＲＸ　ＳＴＩ|WRX STI"))&(df["名称"].str.contains("タイプＳ")), "Model"]  = "WRX STI 2014年モデル Type S"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＷＲＸ ＳＴＩ|ＷＲＸ　ＳＴＩ|WRX STI"))&(df["名称"].str.contains("ＥＪ２０")), "Model"]  = "WRX STI 2014年モデル EJ20 Final Edition"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＷＲＸ ＳＴＩ|ＷＲＸ　ＳＴＩ|WRX STI")), "Model"]  = "WRX STI 2014年モデル ベースグレード"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("インプレッサＷＲＸ|インプレッサWRX"))&(~df["名称"].str.contains("レガシィ")), "Model"]  = "WRX STI 1999年モデル"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("レガシィ"))&(df["Model_year"]>2013)&(df["名称"].str.contains("リミテッド")), "Model"]  = "レガシィ B4 2014年モデル Limited"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("レガシィ"))&(df["Model_year"]>2013), "Model"]  = "レガシィ B4 2014年モデル ベースグレード"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("レガシィ"))&(df["Model_year"]>2008)&(df["Displacement"]>2400), "Model"]  = "レガシィ B4 2009年モデル 2.5i EyeSight"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("レガシィ"))&(df["Model_year"]>2008), "Model"]  = "レガシィ B4 2009年モデル 2.0GT DIT"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("レガシィ"))&(df["Model_year"]>2002)&(df["名称"].str.contains("２．０ＧＴ"))&(df["MT"]==1), "Model"]  = "レガシィ B4 2003年モデル 2.0GT (MT)"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("レガシィ"))&(df["Model_year"]>2002)&(df["名称"].str.contains("２．０ＧＴ")), "Model"]  = "レガシィ B4 2003年モデル 2.0GT"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("レガシィ"))&(df["Model_year"]>2002)&(df["名称"].str.contains("ブリッツェン"))&(df["MT"]==1), "Model"]  = "レガシィ B4 2003年モデル ブリッツェン 2005モデル (MT)"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("レガシィ"))&(df["名称"].str.contains("ブリッツェン")), "Model"]  = "レガシィ B4 2003年モデル ブリッツェン 2005モデル"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("レガシィ"))&(df["Model_year"]>2002)&(df["名称"].str.contains("スペックB"))&(df["MT"]==1), "Model"]  = "レガシィ B4 2003年モデル 2.0GT スペック.B (MT)"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("レガシィ"))&(df["Model_year"]>2002)&(df["名称"].str.contains("スペックB")), "Model"]  = "レガシィ B4 2003年モデル 2.0GT スペック.B"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("レガシィ"))&(df["Model_year"]>2002)&(df["Displacement"]==3000), "Model"] = "レガシィ B4 2003年モデル 3.0R"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("レガシィ"))&(df["Model_year"]>2002)&(df["MT"]==1), "Model"] = "レガシィ B4 2003年モデル 2.0GT (MT)"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("レガシィ"))&(df["Model_year"]>2002), "Model"] = "レガシィ B4 2003年モデル 2.0GT"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("レガシィ"))&(df["Model_year"]<2004)&(df["名称"].str.contains("ＲＳＫ|RSK")), "Model"]  = "レガシィ B4 1998年モデル RSK リミテッドII"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("レガシィ"))&(df["Model_year"]<2003)&(df["Displacement"]==3000), "Model"]  = "レガシィ B4 1998年モデル RS30"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("レガシィ"))&(df["Model_year"]<2003), "Model"]  = "WRX STI 1999年モデル"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("レヴォーグ"))&(df["名称"].str.contains("ＳＴＩ")), "Model"]  = "レヴォーグ 2014年モデル 1.6STI Sport EyeSight"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("レヴォーグ"))&(df["名称"].str.contains("ＧＴ−Ｓ|GT-S")), "Model"]  = "レヴォーグ 2014年モデル 1.6GT-S EyeSight"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("レヴォーグ")), "Model"]  = "レヴォーグ 2014年モデル 1.6GT EyeSight"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("インプレッサ"))&(df["Model_year"]>2015)&(df["Displacement"]>1900)&(df["名称"].str.contains("Ｌ"))&(df["WD"]==4), "Model"]  = "インプレッサ スポーツ 2016年モデル 2.0i-L EyeSight 4WD"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("インプレッサ"))&(df["Model_year"]>2015)&(df["Displacement"]>1900)&(df["名称"].str.contains("Ｌ")), "Model"]  = "インプレッサ スポーツ 2016年モデル 2.0i-L EyeSight"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("インプレッサ"))&(df["Model_year"]>2015)&(df["Displacement"]>1900)&(df["WD"]==4), "Model"]  = "インプレッサ スポーツ 2016年モデル 2.0i-S EyeSight 4WD"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("インプレッサ"))&(df["Model_year"]>2015)&(df["Displacement"]>1900), "Model"]  = "インプレッサ スポーツ 2016年モデル 2.0i-S EyeSight"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("インプレッサ"))&(df["Model_year"]>2015)&(df["Displacement"]<2000)&(df["名称"].str.contains("Ｌ"))&(df["WD"]==4), "Model"]  = "インプレッサ スポーツ 2016年モデル 1.6i-L EyeSight 4WD"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("インプレッサ"))&(df["Model_year"]>2015)&(df["Displacement"]<2000)&(df["名称"].str.contains("Ｌ")), "Model"]  = "インプレッサ スポーツ 2016年モデル 1.6i-L EyeSight"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("インプレッサ"))&(df["Model_year"]>2015)&(df["Displacement"]<2000)&(df["WD"]==4), "Model"]  = "インプレッサ スポーツ 2016年モデル 1.6i-S EyeSight 4WD"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("インプレッサ"))&(df["Model_year"]>2015)&(df["Displacement"]<2000), "Model"]  = "インプレッサ スポーツ 2016年モデル 1.6i-S EyeSight"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("インプレッサ"))&(df["Model_year"]<2016)&(df["Displacement"]>1900), "Model"]  = "インプレッサ スポーツ 2011年モデル 2.0i EyeSight"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("インプレッサ"))&(df["Model_year"]<2016)&(df["Displacement"]<2000)&(df["WD"]==4), "Model"]  = "インプレッサ スポーツ 2011年モデル 1.6i-L 4WD"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("インプレッサ"))&(df["Model_year"]<2016)&(df["Displacement"]<2000), "Model"]  = "インプレッサ スポーツ 2011年モデル 1.6i-L"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＢＲＺ|BRZ"))&(df["Model_year"]>2020), "Model"]  = "BRZ 2021年モデル S (MT)"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＢＲＺ|BRZ"))&(df["名称"].str.contains("ＧＴ|GT")), "Model"]  = "BRZ 2012年モデル GT (MT)"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＢＲＺ|BRZ"))&(df["名称"].str.contains("Ｓ|S")), "Model"]  = "BRZ 2012年モデル S (MT)"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＢＲＺ|BRZ"))&(df["名称"].str.contains("イエロー")), "Model"]  = "BRZ 2012年モデル GT Yellow Edition"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＢＲＺ|BRZ")), "Model"]  = "BRZ 2012年モデル S (MT)"

# レクサス

df.loc[(df["Model"].isnull())&(df["名称"].str.contains("LS500h"))&(df["名称"].str.contains("Fスポーツ"))&(df["WD"]==4), "Model"]  = "LS 2017年モデル LS500h F SPORT AWD"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("LS500h"))&(df["名称"].str.contains("Fスポーツ")), "Model"]  = "LS 2017年モデル LS500h F SPORT"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("LS500h"))&(df["名称"].str.contains("バージョンL"))&(df["WD"]==4), "Model"]  = "LS 2017年モデル LS500h Version L AWD"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("LS500h"))&(df["名称"].str.contains("バージョンL")), "Model"]  = "LS 2017年モデル LS500h Version L"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("LS500h"))&(df["名称"].str.contains("エグゼクティブ"))&(df["WD"]==4), "Model"]  = "LS 2017年モデル LS500h EXECUTIVE AWD"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("LS500h"))&(df["名称"].str.contains("エグゼクティブ")), "Model"]  = "LS 2017年モデル LS500h EXECUTIVE"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("LS500"))&(df["名称"].str.contains("Fスポーツ")), "Model"]  = "LS 2017年モデル LS500 F SPORT"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("LS500"))&(df["名称"].str.contains("バージョンL")), "Model"]  = "LS 2017年モデル LS500 Version L"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＬＳ５００|LS500"))&(df["名称"].str.contains("Iパッケージ"))&(df["WD"]==4), "Model"]  = "LS 2017年モデル LS500 I Package AWD"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＬＳ５００|LS500"))&(df["名称"].str.contains("Iパッケージ")), "Model"]  = "LS 2017年モデル LS500 I Package"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＬＳ５００|LS500"))&(df["名称"].str.contains("エグゼクティブ"))&(df["WD"]==4), "Model"]  = "LS 2017年モデル LS500 EXECUTIVE AWD"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＬＳ５００|LS500"))&(df["名称"].str.contains("エグゼクティブ")), "Model"]  = "LS 2017年モデル LS500 EXECUTIVE"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＬＳ６００ｈ|LS600h"))&(df["名称"].str.contains("バージョンC")), "Model"]  = "LS 2006年モデル LS600h version C"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＬＳ６００ｈ|LS600h")), "Model"]  = "LS 2006年モデル LS600h version L"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＬＳ ４６０|ＬＳ４６０|LS460"))&(df["名称"].str.contains("Ｆスポーツ|Fスポーツ")), "Model"]  = "LS 2006年モデル LS460 F SPORT"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＬＳ ４６０|ＬＳ４６０|LS460"))&(df["名称"].str.contains("バージョンＵ")), "Model"]  = "LS 2006年モデル LS460 version U"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＬＳ ４６０|ＬＳ４６０|LS460"))&(df["名称"].str.contains("４６０Ｌ|バージョンＬ|バージョンL")), "Model"]  = "LS 2006年モデル LS460 version L"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＬＳ ４６０|ＬＳ４６０|LS460"))&(df["名称"].str.contains("バージョンC Iパッケージ")), "Model"]  = "LS 2006年モデル LS460 version C・I package"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＬＳ ４６０|ＬＳ４６０|LS460"))&(df["名称"].str.contains("バージョンC")), "Model"]  = "LS 2006年モデル LS460 version C"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＬＳ ４６０|ＬＳ４６０|LS460")), "Model"]  = "LS 2006年モデル LS460"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＬＳ４３０")), "Model"]  = "LS 2006年モデル LS430"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＬＳ"))&(df["Displacement"]==4600), "Model"]  = "LS 2006年モデル LS460"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＬＳ"))&(df["Displacement"]==5000), "Model"]  = "LS 2006年モデル LS600h"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＩＳ|IS"))&(df["Model_year"]>2012)&(df["Displacement"]>3400)&(df["名称"].str.contains("Ｆスポーツ|Fスポーツ")), "Model"]  = "IS 2013年モデル IS350 F SPORT"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＩＳ|IS"))&(df["Model_year"]>2012)&(df["Displacement"]>3400)&(df["名称"].str.contains("バージョンＬ|バージョンL")), "Model"]  = "IS 2013年モデル IS350 version L"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＩＳ|IS"))&(df["Model_year"]>2012)&(df["Displacement"]>3400), "Model"]  = "IS 2013年モデル IS350"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＩＳ３００ｈ|IS300h"))&(df["Model_year"]>2012)&(df["名称"].str.contains("Ｆスポーツ|Fスポーツ"))&(df["WD"]==4), "Model"]  = "IS 2013年モデル IS300h F SPORT 4WD"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＩＳ３００ｈ|IS300h"))&(df["Model_year"]>2012)&(df["名称"].str.contains("Ｆスポーツ|Fスポーツ")), "Model"]  = "IS 2013年モデル IS300h F SPORT"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＩＳ３００ｈ|IS300h"))&(df["Model_year"]>2012)&(df["名称"].str.contains("バージョンＬ|バージョンL"))&(df["WD"]==4), "Model"]  = "IS 2013年モデル IS300h version L 4WD"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＩＳ３００ｈ|IS300h"))&(df["Model_year"]>2012)&(df["名称"].str.contains("バージョンＬ|バージョンL")), "Model"]  = "IS 2013年モデル IS300h version L"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＩＳ３００ｈ|IS300h"))&(df["Model_year"]>2012)&(df["WD"]==4), "Model"]  = "IS 2013年モデル IS300h 4WD"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＩＳ３００ｈ|IS300h"))&(df["Model_year"]>2012), "Model"]  = "IS 2013年モデル IS300h"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＩＳ３００|IS300"))&(df["Model_year"]>2012)&(df["名称"].str.contains("Ｆスポーツ|Fスポーツ")), "Model"]  = "IS 2013年モデル IS300 F SPORT"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＩＳ３００|IS300"))&(df["Model_year"]>2012)&(df["名称"].str.contains("バージョンＬ|バージョンL")), "Model"]  = "IS 2013年モデル IS300 version L"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＩＳ３００|IS300"))&(df["Model_year"]>2012), "Model"]  = "IS 2013年モデル IS300"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＩＳ|IS"))&(df["Model_year"]>2012)&(df["Displacement"]>2400)&(df["名称"].str.contains("Ｆスポーツ|Fスポーツ"))&(df["WD"]==4), "Model"]  = "IS 2013年モデル IS250 F SPORT 4WD"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＩＳ|IS"))&(df["Model_year"]>2012)&(df["Displacement"]>2400)&(df["名称"].str.contains("Ｆスポーツ|Fスポーツ")), "Model"]  = "IS 2013年モデル IS250 F SPORT"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＩＳ|IS"))&(df["Model_year"]>2012)&(df["Displacement"]>2400)&(df["名称"].str.contains("バージョンＬ|バージョンL"))&(df["WD"]==4), "Model"]  = "IS 2013年モデル IS250 version L 4WD"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＩＳ|IS"))&(df["Model_year"]>2012)&(df["Displacement"]>2400)&(df["名称"].str.contains("バージョンＬ|バージョンL")), "Model"]  = "IS 2013年モデル IS250 version L"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＩＳ|IS"))&(df["Model_year"]>2012)&(df["Displacement"]>2400)&(df["WD"]==4), "Model"]  = "IS 2013年モデル IS250 4WD"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＩＳ|IS"))&(df["Model_year"]>2012)&(df["Displacement"]>2400), "Model"]  = "IS 2013年モデル IS250"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＩＳ|IS"))&(df["Model_year"]>2012)&(df["Displacement"]>1900)&(df["名称"].str.contains("Ｆスポーツ|Fスポーツ")), "Model"]  = "IS 2013年モデル IS200t F SPORT"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＩＳ|IS"))&(df["Model_year"]>2012)&(df["Displacement"]>1900)&(df["名称"].str.contains("バージョンＬ|バージョンL")), "Model"]  = "IS 2013年モデル IS200t version L"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＩＳ|IS"))&(df["Model_year"]>2012)&(df["Displacement"]>1900), "Model"]  = "IS 2013年モデル IS200t"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＩＳ|IS"))&(df["Model_year"]>2012)&(df["名称"].str.contains("IS200t")), "Model"]  = "IS 2013年モデル IS200t F SPORT"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＩＳ|IS"))&(df["Model_year"]<2013)&(df["Displacement"]>3400), "Model"]  = "IS 2005年モデル IS350 version L"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＩＳ|IS"))&(df["Model_year"]<2013)&(df["Displacement"]>2400)&(df["名称"].str.contains("Ｆスポーツ")), "Model"]  = "IS 2005年モデル IS250 F SPORT"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＩＳ|IS"))&(df["Model_year"]<2013)&(df["Displacement"]>2400), "Model"]  = "IS 2005年モデル IS250"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＬＣ|LC"))&(df["名称"].str.contains("コンバーチブル")), "Model"]  = "LC コンバーチブル 2020年モデル LC500 Convertible"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＬＣ|LC"))&(df["名称"].str.contains("500h"))&(df["名称"].str.contains("Lパッケージ")), "Model"]  = "LC 2017年モデル LC500h L Package"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＬＣ|LC"))&(df["名称"].str.contains("500h"))&(df["名称"].str.contains("Sパッケージ")), "Model"]  = "LC 2017年モデル LC500h S Package"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＬＣ|LC"))&(df["名称"].str.contains("500h")), "Model"]  = "LC 2017年モデル LC500h"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＬＣ|LC"))&(df["名称"].str.contains("Lパッケージ")), "Model"]  = "LC 2017年モデル LC500 L Package"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＬＣ|LC"))&(df["名称"].str.contains("Sパッケージ")), "Model"]  = "LC 2017年モデル LC500 S Package"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＬＣ|LC")), "Model"]  = "LC 2017年モデル LC500"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＲＣ")), "Model"]  = "RC F 2014年モデル RC F"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("RC F")), "Model"]  = "RC F 2014年モデル RC F"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("RC350"))&(df["名称"].str.contains("スポーツ")), "Model"]  = "RC 2014年モデル RC350 F SPORT"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("RC350"))&(df["名称"].str.contains("バージョンL")), "Model"]  = "RC 2014年モデル RC350 version L"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("RC350")), "Model"]  = "RC 2014年モデル RC350"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("RC300h"))&(df["名称"].str.contains("スポーツ")), "Model"]  = "RC 2014年モデル RC300h F SPORT"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("RC300h"))&(df["名称"].str.contains("バージョンL")), "Model"]  = "RC 2014年モデル RC300h version L"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("RC300h")), "Model"]  = "RC 2014年モデル RC300h"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("RC300"))&(df["名称"].str.contains("スポーツ")), "Model"]  = "RC 2014年モデル RC300 F SPORT"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("RC300"))&(df["名称"].str.contains("バージョンL")), "Model"]  = "RC 2014年モデル RC300 version L"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("RC300")), "Model"]  = "RC 2014年モデル RC300"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("RC200t"))&(df["名称"].str.contains("スポーツ")), "Model"]  = "RC 2014年モデル RC200t F SPORT"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＥＳ|ES"))&(df["名称"].str.contains("スポーツ")), "Model"]  = "ES 2018年モデル 300h F SPORT"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＥＳ|ES"))&(df["名称"].str.contains("バージョンＬ")), "Model"]  = "ES 2018年モデル 300h Version L"
df.loc[(df["Model"].isnull())&(df["名称"].str.contains("ＥＳ|ES"))&(df["名称"].str.contains("３５０")), "Model"]  = "ES 2018年モデル 300h"

hide_toggle()

"Model"列の欠損値を全て補正しました。  

In [22]:
df["Model"].isnull().sum()

0

In [17]:
df

,名称,価格,色,年式／初度登録,ミッション,全長,駆動方式,全幅,排気量,全高,ドア数,車両重量,乗車定員,燃料,使用燃料,モデル,メーカー,Model_year,Seling_price,Model,Maker,Color,Brightness,MT,WD,Displacement,Door,Capacity,Length,Width,Height,Weight,Limited,Age
0,カムリ ＷＳレザーパッケージ 登録済未使用車 ナビ バックカメラ 黒革,409.8,パールホワイト,令和3年(2021年),CVT,"4,910mm※1",4WD,"1,840mm※1",2500cc,"1,455mm※1",4ドア,"1,680kg※1",5名,ハイブリッド,レギュラー※1,※1 「カムリ 2017年モデル WS レザーパッケージ E-Four」のカタログによる情報...,トヨタ カムリ ＷＳレザーパッケージ 登録済未使用車 ナビ バックカメラ 黒革 409.8万...,2021.0,409.8,カムリ 2017年モデル WS レザーパッケージ E-Four,1.0,3.0,1.0,0,4.0,2500.0,4.0,5.0,4910.0,1840.0,1455.0,1680.0,0.0,1.0
1,カムリ ＷＳ ハイブリッド 衝突被害軽減システム オートクルーズコントロール ＬＥＤヘッドラ...,345.0,プラチナホワイトパールマイカ,令和3年(2021年),AT,"4,910mm※1",2WD,"1,840mm※1",2500cc,"1,445mm※1",4ドア,"1,570kg※1",5名,ハイブリッド,レギュラー※1,※1 「カムリ 2017年モデル WS」のカタログによる情報です。実際の物件状態を保証するも...,トヨタ カムリ ＷＳ ハイブリッド 衝突被害軽減システム オートクルーズコントロール ＬＥＤ...,2021.0,345.0,カムリ 2017年モデル WS,1.0,3.0,1.0,0,2.0,2500.0,4.0,5.0,4910.0,1840.0,1445.0,1570.0,0.0,1.0
2,カムリ Ｘ メモリーナビ バックカメラ 衝突被害軽減システム ＥＴＣ ＬＥＤヘッドランプ ワ...,328.9,シルバーＭ,令和3年(2021年),AT,"4,885mm※1",2WD,"1,840mm※1",2500cc,"1,445mm※1",4ドア,"1,540kg※1",5名,ハイブリッド,レギュラー※1,※1 「カムリ 2017年モデル X」のカタログによる情報です。実際の物件状態を保証するもの...,トヨタ カムリ Ｘ メモリーナビ バックカメラ 衝突被害軽減システム ＥＴＣ ＬＥＤヘッドラ...,2021.0,328.9,カムリ 2017年モデル X,1.0,2.0,0.0,0,2.0,2500.0,4.0,5.0,4885.0,1840.0,1445.0,1540.0,0.0,1.0
3,カムリ ＷＳ ワンオーナー 純正ナビ ドラレコ バックカメラ ヘットアップディスプレイ パノ...,377.0,アティチュードブラックマイカ,令和3年(2021年),CVT,"4,910mm※1",4WD,"1,840mm※1",2500cc,"1,455mm※1",4ドア,"1,660kg※1",5名,ハイブリッド,レギュラー※1,※1 「カムリ 2017年モデル WS E-Four」のカタログによる情報です。実際の物件状...,トヨタ カムリ ＷＳ ワンオーナー 純正ナビ ドラレコ バックカメラ ヘットアップディスプレ...,2021.0,377.0,カムリ 2017年モデル WS E-Four,1.0,0.0,0.0,0,4.0,2500.0,4.0,5.0,4910.0,1840.0,1455.0,1660.0,0.0,1.0
4,カムリ Ｇレザーパッケージ ディーラー保証・走行距離無制限 社用車 サポカーＳ クルーズコン...,360.0,プラチナホワイトパールマイカ,令和3年(2021年),CVT,"4,885mm※1",2WD,"1,840mm※1",2500cc,"1,445mm※1",4ドア,"1,600kg※1",5名,ハイブリッド,レギュラー※1,※1 「カムリ 2017年モデル G レザーパッケージ」のカタログによる情報です。実際の物件...,トヨタ カムリ Ｇレザーパッケージ ディーラー保証・走行距離無制限 社用車 サポカーＳ クル...,2021.0,360.0,カムリ 2017年モデル G レザーパッケージ,1.0,3.0,1.0,0,2.0,2500.0,4.0,5.0,4885.0,1840.0,1445.0,1600.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27491,ＥＳ３００ｈ ３００ｈ バージョンＬ ブラック,548.0,クロ,令和元年(2019年),AT,"4,975mm※1",2WD,"1,865mm※1",2500cc,"1,455mm※1",4ドア,"1,730kg※1",5名,ハイブリッド,レギュラー※1,※1 「ES 2018年モデル 300h Version L」のカタログによる情報です。実際...,レクサス ＥＳ３００ｈ ３００ｈ バージョンＬ ブラック 548.0万円 令和元年(201...,2019.0,548.0,ES 2018年モデル 300h Version L,7.0,0.0,0.0,0,2.0,2500.0,4.0,5.0,4975.0,1865.0,1455.0,1730.0,0.0,3.0
27492,ＥＳ３００ｈ ３００ｈ Ｆスポーツ パール,508.0,パールホワイト,令和元年(2019年),AT,"4,975mm※1",2WD,"1,865mm※1",2500cc,"1,455mm※1",4ドア,"1,720kg※1",5名,ハイブリッド,レギュラー※1,※1 「ES 2018年モデル 300h F SPORT」のカタログによる情報です。実際の物...,レクサス ＥＳ３００ｈ ３００ｈ Ｆスポーツ パール 508.0万円 令和元年(2019年...,2019.0,508.0,ES 2018年モデル 300h F SPORT,7.0,3.0,1.0,0,2.0,2500.0,4.0,5.0,4975.0,1865.0,1455.0,1720.0,0.0,3.0
27493,ＥＳ３００ｈ バージョンＬ,479.0,パールホワイト,令和元年(2019年),AT,"4,975mm※1",2WD,"1,865mm※1",2500cc,"1,455mm※1",4ドア,"1,730kg※1",5名,ハイブリッド,レギュラー※1,※1 「ES 2018年モデル 300h Version L」のカタログによる情報です。実際...,レクサス ＥＳ３００ｈ バージョンＬ 479.0万円 令和元年(2019年) 埼玉県 中古車...,2019.0,479.0,ES 2018年モデル 300h Version L,7.0,3.0,1.0,0,2.0,2500.0,4.0,5.0,4975.0,1865.0,1455.0,1730.0,0.0,3.0
27494,ＥＳ３００ｈ ３００ｈ バージョンＬ サンルーフ／全方,529.8,パールホワイト,平成30年(2018年),AT,,2WD,,2500cc,,4ドア,,5名,ハイブリッド,,※ 掲載しているすべての情報は万全の保証をいたしかねます。,レクサス ＥＳ３００ｈ ３００ｈ バージョンＬ サンルーフ／全方 529.8万円 平成30年...,2018.0,529.8,NaN,7.0,3.0,1.0,0,2.0,2500.0,4.0,5.0,NaN,NaN,NaN,NaN,0.0,4.0


ここで新たにグレード情報を取得します。  
その後中古車両情報にグレード情報を追加し、説明変数に加えます。  
グレード情報をスクレイピングします。  
  
「価格.comグレード」については、以下のリンクをご参照下さい。  
▼グレード一覧ページ  
[価格.com (トヨタ カムリ 2017年モデル) ](https://kakaku.com/item/K0000980810/catalog/)  
▼グレード詳細ページ  
[価格.com (トヨタ カムリ 2017年モデル X) ](https://kakaku.com/item/K0000980810/catalog/GradeID=50579/)

In [107]:
@retry(tries=3, delay=5, backoff=2)
def get_html(url):
    # グレード一覧ページのhtml取得
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    return soup
 
    
def parse_car_detail_info(car_url):
    # 変数定義
    data = {}
    
    # html取得
    soup2 = get_html(car_url)
 
    data["Model"] = soup2.find("h2").getText().strip()
 
    # 物件数
    data["販売数"] = soup2.find("span", {"class": "p-main_link-item-value"}).find("span").getText().strip()
        
    data[soup2.find("th", not("ttl01")).getText().strip()] = soup2.find("td").getText().strip()
    
    th = soup2.findAll("th")
    td = soup2.findAll("td")
    
    # ハイブリッドカーのページには電気情報が含まれているため、それらの要素を削除
    remove3 = [th[0]]
    remove3.append(th[25])
    remove3.append(th[30])
    remove3.append(th[-40])
    remove3.append(th[-32])
    remove3.append(th[-27])
    remove3.append(th[-22])
    remove3.append(th[-11])
    remove3.append(th[-5])
    
    th2 = [i for i in th if i not in remove3]
            
    for i,value in enumerate(th2):
        data[value.getText().strip()] = td[i].getText().strip()
    
    return data
 
def check_next_page(soup):
    if soup.find("li", {"class": "next"}):
        return 1
    else:
        return 0
    
    
# 変数定義
all_data = []
 
# 60ページまでしかページ遷移できないので、地域ごとに分けて収集する
url_list = [
    # トヨタ
    
    "https://kakaku.com/item/K0000980810/catalog/", #カムリ
    "https://kakaku.com/item/K0000295926/catalog/",
    "https://kakaku.com/item/K0000286430/catalog/",
    "https://kakaku.com/item/K0000286429/catalog/",
    "https://kakaku.com/item/K0001120789/catalog/", #スープラ
    "https://kakaku.com/item/K0000286486/catalog/",
    "https://kakaku.com/item/K0001064309/catalog/", #カローラスポーツ
    "https://kakaku.com/item/K0000451834/catalog/", #クラウンアスリート
    "https://kakaku.com/item/K0000286437/catalog/", 
    "https://kakaku.com/item/K0000286436/catalog/",
    "https://kakaku.com/item/K0000286435/catalog/",
    "https://kakaku.com/item/K0000338443/catalog/", #86
    "https://kakaku.com/item/K0001287745/catalog/", #GRヤリス
    "https://kakaku.com/item/K0000287325/catalog/", #マークX
    "https://kakaku.com/item/K0000287324/catalog/",
    
    # ホンダ
    
    "https://kakaku.com/item/K0001374347/catalog/", #シビック
    "https://kakaku.com/item/K0000966426/catalog/",
    "https://kakaku.com/item/K0000286529/catalog/",
    "https://kakaku.com/item/K0000286528/catalog/",
    "https://kakaku.com/item/K0000966416/catalog/", #シビックセダン
    "https://kakaku.com/item/K0000966341/catalog/", #シビックタイプR
    "https://kakaku.com/item/K0000824720/catalog/",
    "https://kakaku.com/item/K0000286531/catalog/",
    "https://kakaku.com/item/K0000286530/catalog/",
    "https://kakaku.com/item/K0000764437/catalog/", #S660
    "https://kakaku.com/item/K0001235866/catalog/", #アコード
    "https://kakaku.com/item/K0000524927/catalog/",
    "https://kakaku.com/item/K0000286515/catalog/",
    "https://kakaku.com/item/K0000286514/catalog/",
    "https://kakaku.com/item/K0000903377/catalog/", #NSX
    "https://kakaku.com/item/K0000286511/catalog/",
    
    # 日産
    
    "https://kakaku.com/item/K0000286512/catalog/", #S2000
    "https://kakaku.com/item/K0000286523/catalog/", #インテグラ
    "https://kakaku.com/item/K0000658496/catalog/", #スカイライン
    "https://kakaku.com/item/K0000286568/catalog/",
    "https://kakaku.com/item/K0000286567/catalog/",
    "https://kakaku.com/item/K0000286596/catalog/", #フーガ
    "https://kakaku.com/item/K0000286595/catalog/",
    "https://kakaku.com/item/K0000286578/catalog/", #フェアレディZ
    "https://kakaku.com/item/K0000286577/catalog/",
    "https://kakaku.com/item/K0000286606/catalog/", #GT-R
    
    # 三菱
    
    "https://kakaku.com/item/K0000286630/catalog/", #ランサーエボリューション
    "https://kakaku.com/item/K0000286629/catalog/",
    
    # マツダ
    
    "https://kakaku.com/item/K0001159813/catalog/", #MAZDA3
    "https://kakaku.com/item/K0000584330/catalog/", #アクセラ
    "https://kakaku.com/item/K0000286677/catalog/",
    "https://kakaku.com/item/K0000286676/catalog/",
    "https://kakaku.com/item/K0000584329/catalog/", #アクセラスポーツ
    "https://kakaku.com/item/K0000287409/catalog/",
    "https://kakaku.com/item/K0000287408/catalog/",
    "https://kakaku.com/item/K0000758192/catalog/", #ロードスター
    "https://kakaku.com/item/K0000286673/catalog/",
    "https://kakaku.com/item/K0000286672/catalog/",
    "https://kakaku.com/item/K0000922633/catalog/", #ロードスターRF
    "https://kakaku.com/item/K0000286674/catalog/", #RX-7
    "https://kakaku.com/item/K0000286675/catalog/", #RX-8
    
    #　スバル
    
    "https://kakaku.com/item/K0000686319/catalog/", #WRX S4
    "https://kakaku.com/item/K0000686320/catalog/", #WRX STI
    "https://kakaku.com/item/K0000287961/catalog/", #WRX STI 4door
    "https://kakaku.com/item/K0000287433/catalog/", #WRX STI 5door
    "https://kakaku.com/item/K0001281697/catalog/", #レヴォーグ
    "https://kakaku.com/item/K0000608487/catalog/",
    "https://kakaku.com/item/K0000709680/catalog/", #レガシィB4
    "https://kakaku.com/item/K0000287218/catalog/",
    "https://kakaku.com/item/K0000287217/catalog/", 
    "https://kakaku.com/item/K0000287216/catalog/",
    "https://kakaku.com/item/K0000914961/catalog/", #インプレッサスポーツ
    "https://kakaku.com/item/K0000319107/catalog/",
    "https://kakaku.com/item/K0001353009/catalog/", #BRZ
    "https://kakaku.com/item/K0000338845/catalog/",
    
    # レクサス
    
    "https://kakaku.com/item/K0001006981/catalog/", #LS
    "https://kakaku.com/item/K0000287042/catalog/",
    "https://kakaku.com/item/K0000510760/catalog/", #IS
    "https://kakaku.com/item/K0000287041/catalog/",
    "https://kakaku.com/item/K0000952538/catalog/", #LC
    "https://kakaku.com/item/K0001267085/catalog/", #LCコンバーチブル
    "https://kakaku.com/item/K0000708217/catalog/", #RC
    "https://kakaku.com/item/K0000708221/catalog/", #RC F
    "https://kakaku.com/item/K0001100392/catalog/", #ES
    
]
 
for url in url_list:
    # 1ページのみスクレイピング
    for page in range(1, 2):
 
        # グレード一覧ページのhtml取得
        soup = get_html(url.format(page))
        
        # 各ページのグレード一覧情報を取得
        for item in soup.findAll("td", {"class": "grade"}):
            # url取得
            car_url = "https://kakaku.com" + item.find("a").get("href")
            
            # 主要諸元情報取得
            all_data.append(parse_car_detail_info(car_url))
            
            # 1秒待つ
            sleep(1)
            
        # 次ページが存在するか確認
        if check_next_page(soup) == 0:
            break
 
# Dataframeに変換
df_spec = pd.DataFrame(all_data)
df_spec.to_pickle("./Specification.pkl")

hide_toggle()

取得したグレード情報はこちらです。

In [19]:
df_spec = pd.read_pickle("./Specification.pkl")
df_spec

,Model,販売数,新車価格,中古車価格,発売日,発売区分,新車販売状況,駆動方式,型式,燃料,動力分類,燃料タンク容量,排気量,トランスミッション,乗車定員,ドア数,シート列数,燃費（WLTCモード）,燃費（WLTCモード|市街地）,燃費（WLTCモード|郊外）,燃費（WLTCモード|高速道路）,WLTC Co2排出量,燃費（JC08モード）,燃費（10.15モード）,ハンドル位置,最小回転半径,最高出力(kW[PS]/rpm),最高トルク(N・m[kgf・m]/rpm),過給機,アイドリングストップ,総電力量(kWh),充電走行距離（km）,最高出力(kW[PS]),最高トルク(N・m[kgf・m]),200V充電時間,急速充電時間,全長,全幅,全高,ホイールベース,最低地上高（未積載時）,車両重量,車両総重量,室内長,室内高,ヘッドルーム 前,ヘッドルーム 後,荷室容量（リアシート立）,荷室容量（リアシート倒）,荷室容量 測定方式,リアシートまでの長さ,ホイールサイズ 前,ホイールサイズ 後,タイヤサイズ 前,タイヤサイズ 後,トレッド幅 前,トレッド幅 後,アルミホイール,スペアタイヤ,パンク修理キット,空気圧警告灯,燃費基準達成率,自動車税減税率,自動車重量税減税率,取得税減税率,環境性能割税率,自動車税,自動車重量税,自動車取得税,環境性能割
0,カムリ 2017年モデル X,36,348万円,,2021年2月1日,マイナーチェンジ,販売中,FF,6AA-AXVH70-AEXDB,レギュラー,ハイブリッド,50L,"2,487cc",-,5名,4ドア,2列,27.1km/L,22.9km/L,30.6km/L,27.1km/L,86.0g/km,,,右,5.7m,131[178]/5700,221[22.5]/3600,なし,標準装備,1kWh,,88[120],202[20.6],,,"4,885mm","1,840mm","1,445mm","2,825mm",145mm,"1,550kg","1,825kg","2,010mm","1,185mm",,,524L,,VDA,,16.0インチ 6.5J,16.0インチ 6.5J,205/65R16,205/65R16,"1,600mm","1,605mm",なし,オプション装備,標準装備,標準装備,,,100%,,非課税,"43,500円","30,000円",,0円
1,カムリ 2017年モデル X E-Four,1,368万円,,2021年2月1日,マイナーチェンジ,販売中,4WD,6AA-AXVH75-AEXDB,レギュラー,ハイブリッド,50L,"2,487cc",-,5名,4ドア,2列,21.6km/L,17.1km/L,24.3km/L,22.5km/L,107.0g/km,24.6km/L,,右,5.7m,131[178]/5700,221[22.5]/3600,なし,標準装備,2kWh,,,,,,"4,885mm","1,840mm","1,455mm","2,825mm",155mm,"1,620kg","1,895kg","2,010mm","1,185mm",,,480L,,VDA,,16.0インチ 6.5J,16.0インチ 6.5J,205/65R16,205/65R16,"1,600mm","1,610mm",なし,なし,標準装備,標準装備,令和2年度燃費基準 + 20%達成,,100%,,非課税,"43,500円","30,000円",,0円
2,カムリ 2017年モデル G,145,379万円,,2021年2月1日,マイナーチェンジ,販売中,FF,6AA-AXVH70-AEXNB,レギュラー,ハイブリッド,50L,"2,487cc",-,5名,4ドア,2列,24.3km/L,20.1km/L,27.2km/L,24.7km/L,96.0g/km,,,右,5.7m,131[178]/5700,221[22.5]/3600,なし,標準装備,1kWh,,88[120],202[20.6],,,"4,885mm","1,840mm","1,445mm","2,825mm",145mm,"1,580kg","1,855kg","2,010mm","1,185mm",,,524L,,VDA,,17.0インチ 7.5J,17.0インチ 7.5J,215/55R17,215/55R17,"1,590mm","1,595mm",標準装備,オプション装備,標準装備,標準装備,,,100%,,非課税,"43,500円","30,000円",,0円
3,カムリ 2017年モデル G E-Four,490,399万円,,2021年2月1日,マイナーチェンジ,販売中,4WD,6AA-AXVH75-AEXNB,レギュラー,ハイブリッド,50L,"2,487cc",-,5名,4ドア,2列,21.6km/L,17.1km/L,24.3km/L,22.5km/L,107.0g/km,24.6km/L,,右,5.7m,131[178]/5700,221[22.5]/3600,なし,標準装備,2kWh,,,,,,"4,885mm","1,840mm","1,455mm","2,825mm",155mm,"1,660kg","1,935kg","2,010mm","1,185mm",,,480L,,VDA,,17.0インチ 7.5J,17.0インチ 7.5J,215/55R17,215/55R17,"1,590mm","1,600mm",標準装備,なし,標準装備,標準装備,令和2年度燃費基準 + 20%達成,,100%,,非課税,"43,500円","30,000円",,0円
4,カムリ 2017年モデル G レザーパッケージ,94,436万円,,2021年2月1日,マイナーチェンジ,販売中,FF,6AA-AXVH70-AEXNB(L),レギュラー,ハイブリッド,50L,"2,487cc",-,5名,4ドア,2列,24.3km/L,20.1km/L,27.2km/L,24.7km/L,96.0g/km,,,右,5.9m,131[178]/5700,221[22.5]/3600,なし,標準装備,1kWh,,88[120],202[20.6],,,"4,885mm","1,840mm","1,445mm","2,825mm",145mm,"1,600kg","1,875kg","2,010mm","1,185mm",,,524L,,VDA,,18.0インチ 8.0J,18.0インチ 8.0J,235/45R18,235/45R18,"1,580mm","1,585mm",標準装備,オプション装備,標準装備,標準装備,,,100%,,非課税,"43,500円","30,000円",,0円
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2488,ES 2018年モデル 300h,12,590万円,,2018年10月24日,フルモデルチェンジ,販売終了,FF,6AA-AXZH10,レギュラー,ハイブリッド,50L,"2,487cc",CVT,5名,4ドア,2列,20.6km/L,16.6km/L,22.7km/L,21.4km/L,113.0g/km,23.4km/L,,右,5.8m,131[178]/5700,221[22.5]/3600,なし,標準装備,10kWh,,88[120],202[20.6],,,"4,975mm","1,865mm","1,455mm","2,870mm",145mm,"1,680kg","1,955kg","2,170mm","1,145mm",995mm,960mm,,,VDA,,17.0インチ 7.5J,17.0インチ 7.5J,215/55R17,215/55R17,"1,600mm","1,610mm",標準装備,オプション装備,標準装備,標準装備,令和2年度燃費基準 + 20%達成,75%,100%,,非課税,"43,500円","30,000円",,0円
2489,ES 2018年モデル 300h F SPORT,106,648万円,,2020年8月6日,マイナーチェンジ,販売終了,FF,6AA-AXZH11-AEXGB,レギュラー,ハイブリッド,50L,"2,487cc",-,5名,4ドア,2列,22.3km/L,18.1km/L,24.7km/L,23.1km/L,,25.1km/L,,右,5.9m,131[178]/5700,221[22.5]/3600,なし,標準装備,2kWh,,88[120],202[20.6],,,"4,975mm","1,865mm","1,455mm","2,870mm",145mm,"1,710kg","1,985kg","2,170mm","1,145mm",995mm,960mm,,,VDA,,19.0イン

このデータには""、"-"が含まれているため、  
それらを欠損値に変換します。  
変換したデータセットを以下に示します。

In [24]:
df_spec.replace(["", "-"], np.nan, inplace=True)
df_spec.isnull().sum()

Model                      0
販売数                        0
新車価格                       0
発売日                        0
発売区分                       0
新車販売状況                    21
駆動方式                      13
型式                        13
燃料                       155
動力分類                      79
排気量                       83
トランスミッション                 82
乗車定員                     154
ドア数                      263
シート列数                    263
最高出力(kW[PS]/rpm)         259
最高トルク(N・m[kgf・m]/rpm)    298
過給機                      606
全長                       160
全幅                       160
全高                       160
ホイールベース                  265
車両重量                     168
ホイールサイズ 前                276
ホイールサイズ 後                276
タイヤサイズ 前                  18
タイヤサイズ 後                  18
自動車税                     263
dtype: int64

価格変動に影響を及ぼさないと思われるデータ、欠損値が多すぎるデータを削除します。  
そして欠損値処理の元となる"型式"情報を"Model"列から推測し補正します。

In [22]:
spec_drop = ["中古車価格", "燃料タンク容量", "燃費（WLTCモード）", "燃費（WLTCモード|市街地）", "燃費（WLTCモード|郊外）","燃費（WLTCモード|高速道路）", "WLTC Co2排出量", "燃費（JC08モード）", "燃費（10.15モード）", "ハンドル位置", "最小回転半径", "アイドリングストップ", "総電力量(kWh)", "充電走行距離（km）", "最高出力(kW[PS])", "最高トルク(N・m[kgf・m])", "200V充電時間", "急速充電時間", "最低地上高（未積載時）", "車両総重量", "室内長", "室内高", "ヘッドルーム 前", "ヘッドルーム 後", "荷室容量（リアシート立）", "荷室容量（リアシート倒）", "荷室容量 測定方式", "リアシートまでの長さ", "トレッド幅 前", "トレッド幅 後", "アルミホイール", "スペアタイヤ", "パンク修理キット", "空気圧警告灯", "燃費基準達成率", "自動車税減税率", "自動車重量税減税率", "取得税減税率", "環境性能割税率", "自動車重量税", "自動車取得税", "環境性能割"]
df_spec = df_spec.drop(spec_drop, axis=1)

df.loc[(df["型式"].isnull())&(df["Model"].str.startswith("86 2012年モデル")), "型式"] = "DBA-ZN6"
df.loc[(df["型式"].isnull())&(df["Model"]=="GRヤリス 2020年モデル RZ First Edition"), "型式"] = "4BA-GXPA16-AGFGZ"
df.loc[(df["型式"].isnull())&(df["Model"]=="GRヤリス 2020年モデル RZ High performance First Edition"), "型式"] = "4BA-GXPA16-AGFGZ(H)"
df.loc[(df["型式"].isnull())&(df["Model"]=="GT-R 2007年モデル GT-R50 by Italdesign"), "型式"] = "4BA-R35"
df.loc[(df["型式"].isnull())&(df["Model"]=="RX-8 2003年モデル マツダスピード バージョン"), "型式"] = "LA-SE3P"
df.loc[(df["型式"].isnull())&(df["Model"]=="レヴォーグ 2014年モデル 1.6GT EyeSight"), "型式"] = "DBA-VM4"

hide_toggle()


In [25]:
df_spec.isnull().sum()

Model                      0
販売数                        0
新車価格                       0
発売日                        0
発売区分                       0
新車販売状況                    21
駆動方式                      13
型式                        13
燃料                       155
動力分類                      79
排気量                       83
トランスミッション                 82
乗車定員                     154
ドア数                      263
シート列数                    263
最高出力(kW[PS]/rpm)         259
最高トルク(N・m[kgf・m]/rpm)    298
過給機                      606
全長                       160
全幅                       160
全高                       160
ホイールベース                  265
車両重量                     168
ホイールサイズ 前                276
ホイールサイズ 後                276
タイヤサイズ 前                  18
タイヤサイズ 後                  18
自動車税                     263
dtype: int64